# Datajoint_sandbox


In [1]:
import pandas as pd
import numpy as np
import os
import datajoint as dj
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime
matplotlib.style.use('ggplot')
%matplotlib inline

<a id='connect'></a>
# Connect to the datajoint00 database
Username and password are same as Princeton login.

**Note**: You need to be on campus or using a VPN to connect to the database.

In [2]:
dj.config['database.host'] = 'datajoint00.pni.princeton.edu'
dj.config['database.user'] = os.environ.get('DJ_DB_USER')
dj.config['database.password'] = os.environ.get('DJ_DB_PASS')
dj.conn()

Connecting ahoag@datajoint00.pni.princeton.edu:3306


DataJoint connection (connected) ahoag@datajoint00.pni.princeton.edu:3306

In [5]:
db_lightsheet = dj.create_virtual_module('ahoag_lightsheet_copy','ahoag_lightsheet_copy')
# db_spockadmin = dj.create_virtual_module('ahoag_spockadmin_demo','ahoag_spockadmin_demo')
# db_lightsheet = dj.create_virtual_module('a','u19lightserv_lightsheet')
# schema = dj.Schema('ahoag_debugjoin')

## Sandbox begins

In [4]:
dj.__version__

'0.13.2'

# Exploring with else case for db transactions

In [12]:
user_insert = {'username':'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa',
              'princeton_email':'a@princeton.edu'}
connection = db_lightsheet.User.connection
try: 
    with connection.transaction:
        db_lightsheet.User().insert1(user_insert)
        localvar=1
except Exception as e:
    print("made it here, local var: ", localvar)

NameError: name 'localvar' is not defined

## Debugging imaging batch dash table
Table is showing individual samples, not the number of batches

In [292]:
imaging_batch_contents = db_lightsheet.Request.ImagingBatch()
imaging_resolution_contents = db_lightsheet.Request.ImagingResolutionRequest()

imaging_request_contents = (clearing_batch_contents * sample_contents * \
    request_contents * imaging_batch_contents).\
        proj('clearer','clearing_progress',
        'imaging_request_date_submitted','imaging_request_time_submitted',
        'imaging_progress','imager','species','number_in_imaging_batch',
        datetime_submitted='TIMESTAMP(imaging_request_date_submitted,imaging_request_time_submitted)',)
''' Figure out how many imaging requests used only Lavision or only Smartspim '''
aggr_imaging_resolution_table = dj.U('username','request_name',
                              'sample_name','imaging_request_number').aggr(
    imaging_resolution_contents,
    all_samples_lavision='SUM(IF(microscope="lavision",1,0))=count(*)',
    all_samples_smartspim='SUM(IF(microscope="smartspim",1,0))=count(*)')
joined_table = imaging_request_contents*aggr_imaging_resolution_table

combined_imaging_contents = dj.U('username','request_name',
    'clearing_batch_number',
    'imaging_batch_number','imaging_request_number').aggr(
    joined_table,
    clearer='MIN(clearer)',
    number_in_batch='COUNT(sample_name)',
    clearing_progress='MIN(clearing_progress)',
    imaging_progress='MIN(imaging_progress)',
    all_samples_cleared='SUM(IF(clearing_progress="complete",1,0))=count(*)')

''' First get all entities that are currently being imaged '''

contents_being_imaged = combined_imaging_contents & 'all_samples_cleared=1' & \
    'imaging_progress="in progress"'

''' Next get all entities that are ready to be imaged '''
contents_ready_to_image = combined_imaging_contents & 'all_samples_cleared=1' & \
 'imaging_progress="incomplete"'
''' Now get all entities on deck (currently being cleared) '''
contents_on_deck = combined_imaging_contents & 'clearing_progress!="complete"' & 'imaging_progress!="complete"'


In [295]:
contents_ready_to_image

username user in the lab,request_name,clearing_batch_number,imaging_batch_number,imaging_request_number,clearer calculated attribute,number_in_batch calculated attribute,clearing_progress calculated attribute,imaging_progress calculated attribute,all_samples_cleared calculated attribute
cz15,zimmerman_03,1,1,1,ll3,2,complete,incomplete,1
cz15,zimmerman_04,1,1,1,ll3,5,complete,incomplete,1
cz15,zimmerman_05,1,1,1,ll3,12,complete,incomplete,1


## Experiment with Table.join(left=True)

In [193]:
@schema
class A(dj.Manual):
    definition = """
    # Test table A
    username : varchar(20)      
    request_name : varchar(20)  
    sample_name : varchar(20)   
    ---
    auditor : varchar(20)       
    """

@schema
class B(dj.Manual):
    definition = """
    # Test table B
    username : varchar(20)      
    request_name : varchar(20)  
    imaging_request_number : tinyint
    ---
    processor: varchar(20)       
    """

In [283]:
dj.Table.update1?

In [180]:
# A_insert_dict = {'username':'abondy',
#                  'request_name':'A323',
#                  'sample_name':'A323-001',
#                  'clearing_batch_number':1,
#                  'imaging_request_number':1,
#                 'auditor':'ejdennis'}
A_insert_dict = {'username':'abondy',
                 'request_name':'A323',
                 'sample_name':'A323-001',
                'auditor':'ejdennis'}
A.insert1(A_insert_dict)

In [218]:
A()

username,request_name,sample_name,auditor
abondy,A323,A323-001,ejdennis


In [219]:
B()

username,request_name,imaging_request_number,processor


In [195]:
A.join(B)

username,request_name,sample_name,imaging_request_number,auditor,processor


In [201]:
dj.Table.join?

In [220]:
dfA = A.fetch(format='frame')
dfB = B.fetch(format='frame')

In [223]:
dfB

,,,processor
username,request_name,imaging_request_number,


In [227]:
dfA.index

MultiIndex([('abondy', 'A323', 'A323-001')],
           names=['username', 'request_name', 'sample_name'])

In [225]:
type(dfB)

pandas.core.frame.DataFrame

In [236]:
pd.merge(dfA,dfB,on=["username","request_name"],how='left')

,,auditor,processor
username,request_name,,
abondy,A323,ejdennis,NaN


In [200]:
A.join(B,left=True,semantic_check=True)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

# Figure out why all_samples page is not showing all of abondy's requests

In [206]:
request_contents = db_lightsheet.Request()
request_contents = request_contents.proj('description','species','number_of_samples',
    'is_archival','auditor',
    datetime_submitted='TIMESTAMP(date_submitted,time_submitted)')
sample_contents = db_lightsheet.Request.Sample()
clearing_batch_contents = db_lightsheet.Request.ClearingBatch()
imaging_request_contents = db_lightsheet.Request.ImagingRequest()
processing_request_contents = db_lightsheet.Request.ProcessingRequest()

replicated_args = dict(number_of_samples='number_of_samples',description='description',
    species='species',datetime_submitted='datetime_submitted',
    clearer='clearer',
    clearing_progress='clearing_progress',
    imager='imager',imaging_progress='imaging_progress',
    is_archival='is_archival',
    link_to_clearing_spreadsheet='link_to_clearing_spreadsheet')

sample_joined_contents = request_contents * sample_contents * clearing_batch_contents

imaging_joined_contents = sample_joined_contents*imaging_request_contents 

processing_joined_contents = imaging_joined_contents.join(processing_request_contents,left=True)

In [249]:
df1 = pd.DataFrame(imaging_joined_contents.fetch(as_dict=True))
df1

,username,request_name,sample_name,clearing_batch_number,imaging_request_number,auditor,description,species,number_of_samples,is_archival,...,perfusion_date,expected_handoff_date,clearer,notes_for_clearer,link_to_clearing_spreadsheet,imager,imaging_request_date_submitted,imaging_request_time_submitted,imaging_performed_date,imaging_progress
0,abondy,A294,A294-001,1,1,None,reconstruction of neuropixels probe tracks (pr...,rat,1,0,...,2021-07-15,2021-07-16,ll3,,None,None,2021-07-15,0 days 13:45:03,2021-08-16,complete
1,abondy,A297,A297-001,1,1,None,reconstruction of probe tracks (two Neuropixel...,rat,1,0,...,2021-08-20,2021-08-23,ll3,,None,None,2021-08-19,0 days 10:06:51,None,incomplete
2,abondy,A323,A323-001,1,1,ejdennis,imaging of CTB (alexfluor 647 conjugated) expr...,rat,1,0,...,2021-05-27,2021-05-28,ll3,good clearing of subcortical tissue particular...,None,None,2021-05-24,0 days 12:27:29,2021-06-15,complete
3,abondy,TS_CTB_3_rats,A288,1,1,None,whole-brain imaging of retrograde tracing expr...,rat,3,0,...,2021-06-10,2021-06-11,ll3,,None,None,2021-06-03,0 days 13:43:00,2021-06-24,complete
4,abondy,TS_CTB_3_rats,A292,1,1,None,whole-brain imaging of retrograde tracing expr...,rat,3,0,...,2021-06-10,2021-06-11,ll3,,None,None,2021-06-03,0 days 13:43:00,2021-06-25,complete
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614,ysagiv,ys_seqpc3_fp,ys_seqpc3_fp-002,1,1,None,Localisation of fibre tips,mouse,4,0,...,2021-07-02,2021-07-12,ll3,,None,None,2021-07-08,0 days 14:28:09,2021-07-15,complete
615,ysagiv,ys_seqpc3_fp,ys_seqpc3_fp-003,1,1,None,Localisation of fibre tips,mouse,4,0,...,2021-07-02,2021-07-12,ll3,,None,None,2021-07-08,0 days 14:28:09,2021-07-15,complete
616,ysagiv,ys_seqpc3_fp,ys_seqpc3_fp-004,1,1,None,Localisation of fibre tips,mouse,4,0,...,2021-07-02,2021-07-12,ll3,,None,None,2021-07-08,0 days 14:28:09,2021-07-15,complete
617,zhihaoz,190912_zz34,sample-001,1,1,None,locating cranial window relative to hippocampa...,mouse,1,1,...,None,None,None,,https://docs.google.com/spreadsheets/d/1A83HVy...,None,2019-09-12,0 days 20:41:42,None,complete


In [210]:
processing_request_contents & {'request_name':'A323'}

username user in the lab,request_name,sample_name,imaging_request_number,processing_request_number,processor user in the lab,processing_request_date_submitted date that the user submitted the request for processing,processing_request_time_submitted time that the user submitted the request for processing,processing_performed_date date that the processing form was submitted by the processor,processing_progress


In [250]:
df2 = pd.DataFrame(processing_request_contents.fetch(as_dict=True))
df2

,username,request_name,sample_name,imaging_request_number,processing_request_number,processor,processing_request_date_submitted,processing_request_time_submitted,processing_performed_date,processing_progress
0,abondy,A297,A297-001,1,1,abondy,2021-08-19,0 days 10:06:51,None,incomplete
1,abondy,TS_CTB_3_rats,A288,1,1,abondy,2021-06-03,0 days 13:43:00,None,incomplete
2,abondy,TS_CTB_3_rats,A292,1,1,abondy,2021-06-03,0 days 13:43:00,None,incomplete
3,abondy,TS_CTB_3_rats,A324,1,1,abondy,2021-06-03,0 days 13:43:00,None,incomplete
4,afalkner,"MFNP2,_MFNP3,_MMNP4,_MMNP5,_MMNP6,_FMNP4,_FMNP...",sample-001,1,1,zmd,2019-11-18,0 days 17:22:11,None,complete
...,...,...,...,...,...,...,...,...,...,...
593,ysagiv,ys_seqpc3_fp,ys_seqpc3_fp-002,1,1,ahoag,2021-07-08,0 days 14:28:09,None,running
594,ysagiv,ys_seqpc3_fp,ys_seqpc3_fp-003,1,1,ysagiv,2021-07-08,0 days 14:28:09,None,running
595,ysagiv,ys_seqpc3_fp,ys_seqpc3_fp-004,1,1,ysagiv,2021-07-08,0 days 14:28:09,None,running
596,zhihaoz,190912_zz34,sample-001,1,1,zmd,2019-09-12,0 days 20:41:42,None,complete


In [252]:
df_merged = pd.merge(df1,df2,how='left',on=["username","request_name","sample_name","imaging_request_number"])
df_merged

,username,request_name,sample_name,clearing_batch_number,imaging_request_number,auditor,description,species,number_of_samples,is_archival,...,imaging_request_date_submitted,imaging_request_time_submitted,imaging_performed_date,imaging_progress,processing_request_number,processor,processing_request_date_submitted,processing_request_time_submitted,processing_performed_date,processing_progress
0,abondy,A294,A294-001,1,1,None,reconstruction of neuropixels probe tracks (pr...,rat,1,0,...,2021-07-15,0 days 13:45:03,2021-08-16,complete,NaN,NaN,NaN,NaT,NaN,NaN
1,abondy,A297,A297-001,1,1,None,reconstruction of probe tracks (two Neuropixel...,rat,1,0,...,2021-08-19,0 days 10:06:51,None,incomplete,1.0,abondy,2021-08-19,0 days 10:06:51,None,incomplete
2,abondy,A323,A323-001,1,1,ejdennis,imaging of CTB (alexfluor 647 conjugated) expr...,rat,1,0,...,2021-05-24,0 days 12:27:29,2021-06-15,complete,NaN,NaN,NaN,NaT,NaN,NaN
3,abondy,TS_CTB_3_rats,A288,1,1,None,whole-brain imaging of retrograde tracing expr...,rat,3,0,...,2021-06-03,0 days 13:43:00,2021-06-24,complete,1.0,abondy,2021-06-03,0 days 13:43:00,None,incomplete
4,abondy,TS_CTB_3_rats,A292,1,1,None,whole-brain imaging of retrograde tracing expr...,rat,3,0,...,2021-06-03,0 days 13:43:00,2021-06-25,complete,1.0,abondy,2021-06-03,0 days 13:43:00,None,incomplete
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614,ysagiv,ys_seqpc3_fp,ys_seqpc3_fp-002,1,1,None,Localisation of fibre tips,mouse,4,0,...,2021-07-08,0 days 14:28:09,2021-07-15,complete,1.0,ahoag,2021-07-08,0 days 14:28:09,None,running
615,ysagiv,ys_seqpc3_fp,ys_seqpc3_fp-003,1,1,None,Localisation of fibre tips,mouse,4,0,...,2021-07-08,0 days 14:28:09,2021-07-15,complete,1.0,ysagiv,2021-07-08,0 days 14:28:09,None,running
616,ysagiv,ys_seqpc3_fp,ys_seqpc3_fp-004,1,1,None,Localisation of fibre tips,mouse,4,0,...,2021-07-08,0 days 14:28:09,2021-07-15,complete,1.0,ysagiv,2021-07-08,0 days 14:28:09,None,running
617,zhihaoz,190912_zz34,sample-001,1,1,None,locating cranial window relative to hippocampa...,mouse,1,1,...,2019-09-12,0 days 20:41:42,None,complete,1.0,zmd,2019-09-12,0 days 20:41:42,None,complete


In [258]:
df_merged.to_dict('records')

[{'username': 'abondy',
  'request_name': 'A294',
  'sample_name': 'A294-001',
  'clearing_batch_number': 1,
  'imaging_request_number': 1,
  'auditor': None,
  'description': 'reconstruction of neuropixels probe tracks (probes dipped in CM-DiI)',
  'species': 'rat',
  'number_of_samples': 1,
  'is_archival': 0,
  'datetime_submitted': Timestamp('2021-07-15 13:45:03'),
  'subject_fullname': '',
  'clearing_protocol': 'iDISCO abbreviated clearing (rat)',
  'antibody1': '',
  'antibody2': '',
  'antibody1_lot': None,
  'antibody2_lot': None,
  'clearing_progress': 'complete',
  'number_in_batch': 1,
  'perfusion_date': datetime.date(2021, 7, 15),
  'expected_handoff_date': datetime.date(2021, 7, 16),
  'clearer': 'll3',
  'notes_for_clearer': '',
  'link_to_clearing_spreadsheet': None,
  'imager': None,
  'imaging_request_date_submitted': datetime.date(2021, 7, 15),
  'imaging_request_time_submitted': Timedelta('0 days 13:45:03'),
  'imaging_performed_date': datetime.date(2021, 8, 16),
 

In [260]:
df_merged[df_merged['request_name'] == 'A294']

,username,request_name,sample_name,clearing_batch_number,imaging_request_number,auditor,description,species,number_of_samples,is_archival,...,imaging_request_date_submitted,imaging_request_time_submitted,imaging_performed_date,imaging_progress,processing_request_number,processor,processing_request_date_submitted,processing_request_time_submitted,processing_performed_date,processing_progress
0,abondy,A294,A294-001,1,1,None,reconstruction of neuropixels probe tracks (pr...,rat,1,0,...,2021-07-15,0 days 13:45:03,2021-08-16,complete,NaN,NaN,NaN,NaT,NaN,NaN


## Figure out what is happening with weird processing requests in all_samples table

In [261]:
all_contents_dict_list = df_merged.to_dict('records')

In [281]:
keep_keys = ['username','request_name','sample_name','species',
             'clearing_protocol','clearer','clearing_progress',
             'antibody1','antibody2',
             'clearing_batch_number','n_imaging_requests',
             'n_processing_requests','datetime_submitted','is_archival',
             'link_to_clearing_spreadsheet']
final_dict_list = []
for d in all_contents_dict_list:
    username = d.get('username')
    request_name = d.get('request_name')
    is_archival = d.get('is_archival')
    current_sample_name = d.get('sample_name')
    imaging_request_number = d.get('imaging_request_number')
    imager = d.get('imager')
    imaging_progress = d.get('imaging_progress')
    imaging_request_dict = {'username':username,'request_name':request_name,
                            'sample_name':current_sample_name,
                            'imaging_request_number':imaging_request_number,
                            'imager':imager,'imaging_progress':imaging_progress,
                            'is_archival':is_archival}
    processing_request_number = d.get('processing_request_number')
    if not np.isnan(processing_request_number):
          processing_request_number = int(processing_request_number)
    processor = d.get('processor')
    processing_progress = d.get('processing_progress')
    processing_request_dict = {'username':username,'request_name':request_name,
                               'sample_name':current_sample_name,
                               'imaging_request_number':imaging_request_number,
                               'processing_request_number':processing_request_number,
                                  'processor':processor,'processing_progress':processing_progress,
                                  'is_archival':is_archival}

    existing_sample_names = [x.get('sample_name') for x in final_dict_list \
                             if x['username']==username and x['request_name']==request_name]
    print(username,request_name,current_sample_name,imaging_request_number,processing_request_number)
    if current_sample_name not in existing_sample_names: # Then new sample, new imaging request, new processing request
        print("new sample")
        new_dict_values = list(map(d.get,keep_keys))
        new_dict = {keep_keys[ii]:new_dict_values[ii] for ii in range(len(keep_keys))}
        new_dict['imaging_requests'] = []
        imaging_request_dict['processing_requests'] = [processing_request_dict]
        new_dict['imaging_requests'].append(imaging_request_dict)
        final_dict_list.append(new_dict)
    else:
        print("existing sample")
        # A repeated sample name could either be
        # a new imaging request or a new processing request at the same imaging request
        # First pull out imaging request dict list  for this request
#         existing_index = existing_sample_names.index(current_sample_name)
        existing_dict = [d for d in final_dict_list if d.get('username') == username and \
                        d.get('request_name') == request_name and \
                        d.get('sample_name') == current_sample_name][0]
#         existing_dict = final_dict_list[existing_index]
        existing_imaging_request_dicts = existing_dict['imaging_requests']
        existing_imaging_request_numbers = [x.get('imaging_request_number') for x in existing_imaging_request_dicts]
        if imaging_request_number not in existing_imaging_request_numbers: # Then its a new imaging request and processing request
            print("new imaging request")
            imaging_request_dict['processing_requests'] = [processing_request_dict]
            existing_dict['imaging_requests'].append(imaging_request_dict)
        else: # Then it's an old imaging request and new processing request
            print("same imaging request, new processing request")
            existing_imaging_request_index = existing_imaging_request_numbers.index(imaging_request_number)
            existing_imaging_request_dict = existing_imaging_request_dicts[existing_imaging_request_index]
            existing_imaging_request_dict['processing_requests'].append(processing_request_dict)


abondy A294 A294-001 1 nan
new sample
abondy A297 A297-001 1 1
new sample
abondy A323 A323-001 1 nan
new sample
abondy TS_CTB_3_rats A288 1 1
new sample
abondy TS_CTB_3_rats A292 1 1
new sample
abondy TS_CTB_3_rats A324 1 1
new sample
afalkner MFNP2,_MFNP3,_MMNP4,_MMNP5,_MMNP6,_FMNP4,_FMNP5,_FMNP6 sample-001 1 1
new sample
afalkner MFNP2,_MFNP3,_MMNP4,_MMNP5,_MMNP6,_FMNP4,_FMNP5,_FMNP6 sample-002 1 1
new sample
afalkner MFNP2,_MFNP3,_MMNP4,_MMNP5,_MMNP6,_FMNP4,_FMNP5,_FMNP6 sample-003 1 1
new sample
afalkner MFNP2,_MFNP3,_MMNP4,_MMNP5,_MMNP6,_FMNP4,_FMNP5,_FMNP6 sample-004 1 1
new sample
afalkner MFNP2,_MFNP3,_MMNP4,_MMNP5,_MMNP6,_FMNP4,_FMNP5,_FMNP6 sample-005 1 1
new sample
afalkner MFNP2,_MFNP3,_MMNP4,_MMNP5,_MMNP6,_FMNP4,_FMNP5,_FMNP6 sample-006 1 1
new sample
afalkner MFNP2,_MFNP3,_MMNP4,_MMNP5,_MMNP6,_FMNP4,_FMNP5,_FMNP6 sample-007 1 1
new sample
afalkner MFNP2,_MFNP3,_MMNP4,_MMNP5,_MMNP6,_FMNP4,_FMNP5,_FMNP6 sample-008 1 1
new sample
aichen zimmerman_01_LaVision zimmerman_01_LaV

oostland Oostland_M96_M101 Oostland_M96 1 1
new sample
pbibawi 20190405_pb_X015,_20190405_pb_X045,_20190405_pb_A233,20190405_pb sample-001 1 1
new sample
pbibawi 20190405_pb_X015,_20190405_pb_X045,_20190405_pb_A233,20190405_pb sample-002 1 1
new sample
pbibawi 20191001_pb_Z265_ctb647_rPPC,_20191001_pb_Z266_ctb647_lPPC,_2019 sample-001 1 1
new sample
pbibawi 20191001_pb_Z265_ctb647_rPPC,_20191001_pb_Z266_ctb647_lPPC,_2019 sample-002 1 1
new sample
pbibawi 20191001_pb_Z265_ctb647_rPPC,_20191001_pb_Z266_ctb647_lPPC,_2019 sample-003 1 1
new sample
pbibawi 20191001_pb_Z265_ctb647_rPPC,_20191001_pb_Z266_ctb647_lPPC,_2019 sample-005 1 1
new sample
pbibawi 20191001_pb_Z265_ctb647_rPPC,_20191001_pb_Z266_ctb647_lPPC,_2019 z268 1 1
new sample
pbibawi pb_udisco pb_udisco-E155 1 nan
new sample
pbibawi pb_udisco pb_udisco_M122 1 nan
new sample
pbibawi pb_udisco pb_udisco_X073 1 nan
new sample
pbibawi pb_udisco pb_udisco_X077 1 nan
new sample
pbibawi pb_udisco pb_udisco_X078 1 nan
new sample
pbibawi 

In [282]:
final_dict_list

[{'username': 'abondy',
  'request_name': 'A294',
  'sample_name': 'A294-001',
  'species': 'rat',
  'clearing_protocol': 'iDISCO abbreviated clearing (rat)',
  'clearer': 'll3',
  'clearing_progress': 'complete',
  'antibody1': '',
  'antibody2': '',
  'clearing_batch_number': 1,
  'n_imaging_requests': None,
  'n_processing_requests': None,
  'datetime_submitted': Timestamp('2021-07-15 13:45:03'),
  'is_archival': 0,
  'link_to_clearing_spreadsheet': None,
  'imaging_requests': [{'username': 'abondy',
    'request_name': 'A294',
    'sample_name': 'A294-001',
    'imaging_request_number': 1,
    'imager': None,
    'imaging_progress': 'complete',
    'is_archival': 0,
    'processing_requests': [{'username': 'abondy',
      'request_name': 'A294',
      'sample_name': 'A294-001',
      'imaging_request_number': 1,
      'processing_request_number': nan,
      'processor': nan,
      'processing_progress': nan,
      'is_archival': 0}]}]},
 {'username': 'abondy',
  'request_name': 'A2

## Figure out why in demo database zimmerman_02 request shows a null value for datetime_submitted

In [106]:
request_contents = db_lightsheet.Request()
clearing_batch_contents = db_lightsheet.Request.ClearingBatch()
imaging_request_contents = db_lightsheet.Request.ImagingRequest()
processing_request_contents = db_lightsheet.Request.ProcessingRequest()

''' Now figure out what fraction of the samples in each request are cleared/imaged/processed '''    
replicated_args = dict(is_archival='is_archival',number_of_samples='number_of_samples',description='description',
    species='species',datetime_submitted='datetime_submitted')

request_joined_contents = dj.U('username','request_name').aggr(
    request_contents.join(clearing_batch_contents,left=True),
    is_archival='MIN(is_archival)',
    number_of_samples='MIN(number_of_samples)',
    number_in_batch='MIN(number_in_batch)',
    description='MIN(description)',
    species='MIN(species)',
    datetime_submitted='TIMESTAMP(MIN(date_submitted),MIN(time_submitted))',
    n_cleared='CONVERT(SUM(IF(clearing_progress="complete",number_in_batch,0)),char)').proj(
    **replicated_args,
        fraction_cleared='CONCAT(n_cleared,"/",CONVERT(number_of_samples,char))')

imaging_aggr_contents = dj.U('username','request_name').aggr(
    imaging_request_contents,
    n_imaged='CONVERT(SUM(imaging_progress="complete"),char)',
    total_imaging_requests='CONVERT(COUNT(*),char)',
    )
imaging_joined_contents = (request_joined_contents.join(imaging_aggr_contents,left=True)).proj(**replicated_args,
    fraction_cleared='fraction_cleared',
    fraction_imaged='IF(n_imaged is NULL,"0/0",CONCAT(n_imaged,"/",total_imaging_requests))' 
    )
processing_aggr_contents = dj.U('username','request_name').aggr(
    processing_request_contents,
    n_processed='CONVERT(SUM(processing_progress="complete"),char)',
    total_processing_requests='CONVERT(COUNT(processing_progress),char)',
    )
processing_joined_contents = (imaging_joined_contents.join(processing_aggr_contents,left=True)).proj(
    **replicated_args,
    fraction_cleared='fraction_cleared',
    fraction_imaged='fraction_imaged',
    fraction_processed='IF(n_processed is NULL,"0/0",CONCAT(n_processed,"/",total_processing_requests))' 
    ) 

# Figure out why all_requests page is not showing all of abondy's requests

In [149]:
request_contents = db_lightsheet.Request()
clearing_batch_contents = db_lightsheet.Request.ClearingBatch()
imaging_request_contents = db_lightsheet.Request.ImagingRequest()
processing_request_contents = db_lightsheet.Request.ProcessingRequest()

# logger.debug(request_contents)
''' Now figure out what fraction of the samples in each request are cleared/imaged/processed '''    
replicated_args = dict(is_archival='is_archival',number_of_samples='number_of_samples',description='description',
    species='species',datetime_submitted='datetime_submitted')

request_joined_contents = dj.U('username','request_name').aggr(
    request_contents * clearing_batch_contents,
    is_archival='MIN(is_archival)',
    number_of_samples='MIN(number_of_samples)',
    number_in_batch='MIN(number_in_batch)',
    description='MIN(description)',
    species='MIN(species)',
    datetime_submitted='TIMESTAMP(MIN(date_submitted),MIN(time_submitted))',
    n_cleared='CONVERT(SUM(IF(clearing_progress="complete",number_in_batch,0)),char)').proj(
    **replicated_args,
        fraction_cleared='CONCAT(n_cleared,"/",CONVERT(number_of_samples,char))')
# A problem arises going from request joined contents to imaging_joined_contents - we lose some requests
# It is because there is not an imaging request for every request 
# Strangely there is an imaging request for each of abondy's requests, but we lose those somehow
imaging_aggr_contents = dj.U('username','request_name').aggr(
    imaging_request_contents,
    n_imaged='CONVERT(SUM(imaging_progress="complete"),char)',
    total_imaging_requests='CONVERT(COUNT(*),char)',
    )
imaging_joined_contents = (request_joined_contents*imaging_aggr_contents).proj(**replicated_args,
    fraction_cleared='fraction_cleared',
    fraction_imaged='IF(n_imaged is NULL,"0/0",CONCAT(n_imaged,"/",total_imaging_requests))' 
    )
processing_aggr_contents = dj.U('username','request_name').aggr(
    processing_request_contents,
    n_processed='CONVERT(SUM(processing_progress="complete"),char)',
    total_processing_requests='CONVERT(COUNT(processing_progress),char)',
    )
processing_joined_contents = (imaging_joined_contents*processing_aggr_contents).proj(
    **replicated_args,
    fraction_cleared='fraction_cleared',
    fraction_imaged='fraction_imaged',
    fraction_processed='IF(n_processed is NULL,"0/0",CONCAT(n_processed,"/",total_processing_requests))' 
    ) 

In [71]:
imaging_joined_contents*processing_aggr_contents & {'username':'abondy'}

username user in the lab,request_name,is_archival calculated attribute,number_of_samples calculated attribute,description calculated attribute,species calculated attribute,datetime_submitted calculated attribute,fraction_cleared calculated attribute,fraction_imaged calculated attribute,n_processed calculated attribute,total_processing_requests calculated attribute
abondy,A297,0,1,reconstruction of probe tracks (two Neuropixels probes dipped in CM-DiI). no need for single-cell resolution.,rat,2021-08-19 10:06:51,0/1,0/1,0,1
abondy,TS_CTB_3_rats,0,3,whole-brain imaging of retrograde tracing expression (CTB conjugated to alexafluor 647),rat,2021-06-03 13:43:00,3/3,3/3,0,3


In [150]:
imaging_joined_contents & {'username':'abondy'}

username user in the lab,request_name,is_archival calculated attribute,number_of_samples calculated attribute,description calculated attribute,species calculated attribute,datetime_submitted calculated attribute,fraction_cleared calculated attribute,fraction_imaged calculated attribute
abondy,A294,0,1,reconstruction of neuropixels probe tracks (probes dipped in CM-DiI),rat,2021-07-15 13:45:03,1/1,1/1
abondy,A297,0,1,reconstruction of probe tracks (two Neuropixels probes dipped in CM-DiI). no need for single-cell resolution.,rat,2021-08-19 10:06:51,0/1,0/1
abondy,A323,0,1,"imaging of CTB (alexfluor 647 conjugated) expression throughout rat brain, used for retrograde tracing from posterior striatum. Emily suggested uDisco would be better than iDisco for this application.",rat,2021-05-24 12:27:29,1/1,1/1
abondy,TS_CTB_3_rats,0,3,whole-brain imaging of retrograde tracing expression (CTB conjugated to alexafluor 647),rat,2021-06-03 13:43:00,3/3,3/3


In [151]:
processing_aggr_contents & {'username':'abondy'}

username user in the lab,request_name,n_processed calculated attribute,total_processing_requests calculated attribute
abondy,A297,0,1
abondy,TS_CTB_3_rats,0,3


In [152]:
(imaging_joined_contents & {'username':'abondy'}).join(processing_aggr_contents & {'username':'abondy'},left=True)

username user in the lab,request_name,is_archival calculated attribute,number_of_samples calculated attribute,description calculated attribute,species calculated attribute,datetime_submitted calculated attribute,fraction_cleared calculated attribute,fraction_imaged calculated attribute,n_processed calculated attribute,total_processing_requests calculated attribute
abondy,A294,0,1,reconstruction of neuropixels probe tracks (probes dipped in CM-DiI),rat,2021-07-15 13:45:03,1/1,1/1,None,None
abondy,A297,0,1,reconstruction of probe tracks (two Neuropixels probes dipped in CM-DiI). no need for single-cell resolution.,rat,2021-08-19 10:06:51,0/1,0/1,0,1
abondy,A323,0,1,"imaging of CTB (alexfluor 647 conjugated) expression throughout rat brain, used for retrograde tracing from posterior striatum. Emily suggested uDisco would be better than iDisco for this application.",rat,2021-05-24 12:27:29,1/1,1/1,None,None
abondy,TS_CTB_3_rats,0,3,whole-brain imaging of retrograde tracing expression (CTB conjugated to alexafluor 647),rat,2021-06-03 13:43:00,3/3,3/3,0,3


In [8]:
imaging_joined_contents

username user in the lab,request_name,is_archival calculated attribute,number_of_samples calculated attribute,description calculated attribute,species calculated attribute,datetime_submitted calculated attribute,fraction_cleared calculated attribute,fraction_imaged calculated attribute
abondy,A294,0,1,reconstruction of neuropixels probe tracks (probes dipped in CM-DiI),rat,2021-07-15 13:45:03,1/1,1/1
abondy,A297,0,1,reconstruction of probe tracks (two Neuropixels probes dipped in CM-DiI). no need for single-cell resolution.,rat,2021-08-19 10:06:51,0/1,0/1
abondy,A323,0,1,"imaging of CTB (alexfluor 647 conjugated) expression throughout rat brain, used for retrograde tracing from posterior striatum. Emily suggested uDisco would be better than iDisco for this application.",rat,2021-05-24 12:27:29,1/1,1/1
abondy,TS_CTB_3_rats,0,3,whole-brain imaging of retrograde tracing expression (CTB conjugated to alexafluor 647),rat,2021-06-03 13:43:00,3/3,3/3
afalkner,"MFNP2,_MFNP3,_MMNP4,_MMNP5,_MMNP6,_FMNP4,_FMNP5,_FMNP6",1,8,"Fos induction, SW mice",mouse,2019-11-18 17:22:11,8/8,8/8
aichen,zimmerman_01_LaVision,0,1,to image c-fos stained mouse brains on LaVision,mouse,2020-12-11 11:38:20,1/1,1/1
apv2,20190313_IBL_DiI_1,1,1,Detection of CM DiI,mouse,2019-03-13 01:43:03,1/1,1/1
apv2,ibl_witten_04,1,1,Detection of electrode tracks CM-DiI,mouse,2019-08-12 15:07:23,1/1,1/1
cz15,zimmerman_01,0,2,whole-brain Fos staining,mouse,2020-10-27 10:10:45,2/2,2/2
cz15,zimmerman_02,0,12,"Clearing, Fos staining, and smartSPIM imaging for twelve brains.",mouse,2021-03-03 15:53:40,12/12,25/25


In [7]:
processing_joined_contents

username user in the lab,request_name,is_archival calculated attribute,number_of_samples calculated attribute,description calculated attribute,species calculated attribute,datetime_submitted calculated attribute,fraction_cleared calculated attribute,fraction_imaged calculated attribute,fraction_processed calculated attribute
abondy,A297,0,1,reconstruction of probe tracks (two Neuropixels probes dipped in CM-DiI). no need for single-cell resolution.,rat,2021-08-19 10:06:51,0/1,0/1,0/1
abondy,TS_CTB_3_rats,0,3,whole-brain imaging of retrograde tracing expression (CTB conjugated to alexafluor 647),rat,2021-06-03 13:43:00,3/3,3/3,0/3
afalkner,"MFNP2,_MFNP3,_MMNP4,_MMNP5,_MMNP6,_FMNP4,_FMNP5,_FMNP6",1,8,"Fos induction, SW mice",mouse,2019-11-18 17:22:11,8/8,8/8,8/8
aichen,zimmerman_01_LaVision,0,1,to image c-fos stained mouse brains on LaVision,mouse,2020-12-11 11:38:20,1/1,1/1,0/1
apv2,20190313_IBL_DiI_1,1,1,Detection of CM DiI,mouse,2019-03-13 01:43:03,1/1,1/1,1/1
apv2,ibl_witten_04,1,1,Detection of electrode tracks CM-DiI,mouse,2019-08-12 15:07:23,1/1,1/1,1/1
cz15,zimmerman_01,0,2,whole-brain Fos staining,mouse,2020-10-27 10:10:45,2/2,2/2,0/2
cz15,zimmerman_02,0,12,"Clearing, Fos staining, and smartSPIM imaging for twelve brains.",mouse,2021-03-03 15:53:40,12/12,25/25,0/25
cz15,zimmerman_03,0,2,visualize tract of chronically implanted Neuropixels probes (coated with DiI before implantation),mouse,2021-07-20 14:27:24,2/2,0/2,0/2
cz15,zimmerman_04,0,5,brainwide Fos staining + mapping,mouse,2021-07-20 14:32:30,5/5,0/5,0/5


## Figure out why request_overview for requests with no processing requests are showing no samples

In [58]:
username='abondy'
request_name='A323'
request_contents = db_lightsheet.Request() & f'request_name="{request_name}"' & \
        f'username="{username}"'
request_contents = request_contents.proj('description','species','number_of_samples',
    'is_archival',datetime_submitted='TIMESTAMP(date_submitted,time_submitted)')
sample_contents = db_lightsheet.Request.Sample() & f'request_name="{request_name}"' & f'username="{username}"' 

clearing_batch_contents = db_lightsheet.Request.ClearingBatch() & \
    f'request_name="{request_name}"' & f'username="{username}"' 
clearing_batch_sample_contents = db_lightsheet.Request.ClearingBatchSample() & \
    f'request_name="{request_name}"' & f'username="{username}"' 
imaging_request_contents = db_lightsheet.Request.ImagingRequest() & \
    f'request_name="{request_name}"' & f'username="{username}"' 
processing_request_contents = db_lightsheet.Request.ProcessingRequest() & \
    f'request_name="{request_name}"' & f'username="{username}"' 

replicated_args = dict(number_of_samples='number_of_samples',description='description',
    species='species',clearing_progress='clearing_progress',
    clearer='clearer',
    imaging_progress='imaging_progress',imager='imager',
    is_archival='is_archival',
    link_to_clearing_spreadsheet='link_to_clearing_spreadsheet')
sample_joined_contents = request_contents * clearing_batch_sample_contents * clearing_batch_contents.proj(
'clearing_progress','clearer','link_to_clearing_spreadsheet')
imaging_aggr_contents = dj.U('username','request_name','sample_name').aggr(
    imaging_request_contents,
    imaging_progress='MIN(imaging_progress)',
    imager='MIN(imager)',
    imaging_request_number='MIN(imaging_request_number)',
    total_imaging_requests='COUNT(*)',
    n_imaged='CONVERT(SUM(imaging_progress="complete"),char)',).proj(**replicated_args,
           total_imaging_requests='IF(n_imaged is NULL, "0",total_imaging_requests)',
           imaging_request_number='IF(imaging_request_number is NULL, "N/A",imaging_request_number)')

imaging_joined_contents = sample_joined_contents * imaging_aggr_contents

processing_aggr_contents = dj.U('username','request_name').aggr(   
    processing_request_contents,
    processing_request_number='MIN(processing_request_number)',
    processor='MIN(processor)',
    processing_progress='MIN(processing_progress)',
    n_processed='CONVERT(SUM(processing_progress="complete"),char)',
    total_processing_requests='CONVERT(COUNT(processing_progress),char)')

if len(processing_aggr_contents) == 0:
    processing_joined_contents = imaging_joined_contents.proj(
        **replicated_args,
        imaging_request_number='imaging_request_number',
        processing_request_number='IF(imaging_request_number*0,"N/A","N/A")',
        processor='IF(imaging_request_number*0,"N/A","N/A")',
        processing_progress='IF(imaging_request_number*0,"N/A","N/A")',
        total_imaging_requests='total_imaging_requests',
        total_processing_requests='IF(imaging_request_number*0,0,0)',
    )
else:
    processing_joined_contents = (imaging_joined_contents * processing_aggr_contents).proj(
        **replicated_args,
        imaging_request_number='imaging_request_number',
        processing_request_number='IF(processing_request_number is NULL, "N/A",processing_request_number)',
        processor='processor',
        processing_progress='processing_progress',
        total_imaging_requests='total_imaging_requests',
        total_processing_requests='IF(n_processed is NULL,0,total_processing_requests)', 
    )

In [59]:
processing_joined_contents

username user in the lab,request_name,sample_name,clearing_batch_number,description,species,number_of_samples,is_archival,clearing_progress,clearer user in the lab,link_to_clearing_spreadsheet,imaging_progress calculated attribute,imager calculated attribute,total_imaging_requests calculated attribute,imaging_request_number calculated attribute,processing_request_number calculated attribute,processor calculated attribute,processing_progress calculated attribute,total_processing_requests calculated attribute
abondy,A323,A323-001,1,"imaging of CTB (alexfluor 647 conjugated) expression throughout rat brain, used for retrograde tracing from posterior striatum. Emily suggested uDisco would be better than iDisco for this application.",rat,1,0,complete,ll3,None,complete,None,1,1,N/A,N/A,N/A,0


In [31]:
aprocessing_aggr_contents

username user in the lab,request_name,processing_request_number calculated attribute,processor calculated attribute,processing_progress calculated attribute,n_processed calculated attribute,total_processing_requests calculated attribute


In [32]:
len(processing_aggr_contents)

0

In [24]:
imaging_joined_contents

username user in the lab,request_name,sample_name,clearing_batch_number,description,species,number_of_samples,is_archival,datetime_submitted calculated attribute,clearing_protocol,antibody1,antibody2,clearing_progress,clearer user in the lab,link_to_clearing_spreadsheet,imaging_progress calculated attribute,imager calculated attribute,total_imaging_requests calculated attribute,imaging_request_number calculated attribute
abondy,A323,A323-001,1,"imaging of CTB (alexfluor 647 conjugated) expression throughout rat brain, used for retrograde tracing from posterior striatum. Emily suggested uDisco would be better than iDisco for this application.",rat,1,0,2021-05-24 12:27:29,uDISCO (rat),,,complete,ll3,None,complete,None,1,1


In [29]:
processing_aggr_contents

username user in the lab,request_name,processing_request_number calculated attribute,processor calculated attribute,processing_progress calculated attribute,n_processed calculated attribute,total_processing_requests calculated attribute


## Figure out why all_samples is not showing up for abondy A294 request

In [60]:
request_contents = db_lightsheet.Request()
request_contents = request_contents.proj('description','species','number_of_samples',
    'is_archival','auditor',
    datetime_submitted='TIMESTAMP(date_submitted,time_submitted)')
sample_contents = db_lightsheet.Request.Sample()
clearing_batch_contents = db_lightsheet.Request.ClearingBatch()
imaging_request_contents = db_lightsheet.Request.ImagingRequest()
processing_request_contents = db_lightsheet.Request.ProcessingRequest()

replicated_args = dict(number_of_samples='number_of_samples',description='description',
    species='species',datetime_submitted='datetime_submitted',
    clearer='clearer',
    clearing_progress='clearing_progress',
    imager='imager',imaging_progress='imaging_progress',
    is_archival='is_archival',
    link_to_clearing_spreadsheet='link_to_clearing_spreadsheet')

sample_joined_contents = request_contents * sample_contents * clearing_batch_contents

imaging_joined_contents = sample_joined_contents * imaging_request_contents 

processing_joined_contents = imaging_joined_contents * processing_request_contents

In [62]:
processing_joined_contents & {'username':'abondy'}

username user in the lab,request_name,sample_name,clearing_batch_number,imaging_request_number,processing_request_number,auditor user in the lab,description,species,number_of_samples,is_archival,datetime_submitted calculated attribute,subject_fullname,clearing_protocol,antibody1,antibody2,antibody1_lot,antibody2_lot,clearing_progress,number_in_batch,perfusion_date,expected_handoff_date,clearer user in the lab,notes_for_clearer,link_to_clearing_spreadsheet,imager user in the lab,imaging_request_date_submitted date that the user submitted the request for imaging,imaging_request_time_submitted time that the user submitted the request for imaging,imaging_performed_date date that the imaging form was submitted by the imager,imaging_progress,processor user in the lab,processing_request_date_submitted date that the user submitted the request for processing,processing_request_time_submitted time that the user submitted the request for processing,processing_performed_date date that the processing form was submitted by the processor,processing_progress
abondy,A297,A297-001,1,1,1,None,reconstruction of probe tracks (two Neuropixels probes dipped in CM-DiI). no need for single-cell resolution.,rat,1,0,2021-08-19 10:06:51,,iDISCO abbreviated clearing (rat),,,None,None,in progress,1,2021-08-20,2021-08-23,ll3,,None,None,2021-08-19,10:06:51,None,incomplete,abondy,2021-08-19,10:06:51,None,incomplete
abondy,TS_CTB_3_rats,A288,1,1,1,None,whole-brain imaging of retrograde tracing expression (CTB conjugated to alexafluor 647),rat,3,0,2021-06-03 13:43:00,,uDISCO (rat),,,None,None,complete,3,2021-06-10,2021-06-11,ll3,,None,None,2021-06-03,13:43:00,2021-06-24,complete,abondy,2021-06-03,13:43:00,None,incomplete
abondy,TS_CTB_3_rats,A292,1,1,1,None,whole-brain imaging of retrograde tracing expression (CTB conjugated to alexafluor 647),rat,3,0,2021-06-03 13:43:00,,uDISCO (rat),,,None,None,complete,3,2021-06-10,2021-06-11,ll3,,None,None,2021-06-03,13:43:00,2021-06-25,complete,abondy,2021-06-03,13:43:00,None,incomplete
abondy,TS_CTB_3_rats,A324,1,1,1,None,whole-brain imaging of retrograde tracing expression (CTB conjugated to alexafluor 647),rat,3,0,2021-06-03 13:43:00,,uDISCO (rat),,,None,None,complete,3,2021-06-10,2021-06-11,ll3,,None,None,2021-06-03,13:43:00,2021-06-29,complete,abondy,2021-06-03,13:43:00,None,incomplete


## Figure out why request_overview is not showing any samples for Abondy 294 request

'A323'

In [86]:
username='abondy'
request_name = "A297"
request_contents = db_lightsheet.Request() & f'request_name="{request_name}"' & \
        f'username="{username}"'
request_contents = request_contents.proj('description','species','number_of_samples',
    'is_archival',datetime_submitted='TIMESTAMP(date_submitted,time_submitted)')
sample_contents = db_lightsheet.Request.Sample() & f'request_name="{request_name}"' & f'username="{username}"' 

clearing_batch_contents = db_lightsheet.Request.ClearingBatch() & \
    f'request_name="{request_name}"' & f'username="{username}"' 
clearing_batch_sample_contents = db_lightsheet.Request.ClearingBatchSample() & \
    f'request_name="{request_name}"' & f'username="{username}"' 
imaging_request_contents = db_lightsheet.Request.ImagingRequest() & \
    f'request_name="{request_name}"' & f'username="{username}"' 
processing_request_contents = db_lightsheet.Request.ProcessingRequest() & \
    f'request_name="{request_name}"' & f'username="{username}"' 

replicated_args = dict(number_of_samples='number_of_samples',description='description',
    species='species',clearing_progress='clearing_progress',
    clearer='clearer',
    imaging_progress='imaging_progress',imager='imager',
    is_archival='is_archival',
    link_to_clearing_spreadsheet='link_to_clearing_spreadsheet')

sample_joined_contents = request_contents * clearing_batch_sample_contents * clearing_batch_contents.proj(
'clearing_progress','clearer','link_to_clearing_spreadsheet')
imaging_aggr_contents = dj.U('username','request_name','sample_name').aggr(
    imaging_request_contents,
    imaging_progress='MIN(imaging_progress)',
    imager='MIN(imager)',
    imaging_request_number='MIN(imaging_request_number)',
    total_imaging_requests='COUNT(*)',
    n_imaged='CONVERT(SUM(imaging_progress="complete"),char)',).proj(**replicated_args,
           total_imaging_requests='IF(n_imaged is NULL, "0",total_imaging_requests)',
           imaging_request_number='IF(imaging_request_number is NULL, "N/A",imaging_request_number)')

 

In [87]:
processing_joined_contents & {'username':'abondy'}

username user in the lab,request_name,sample_name,clearing_batch_number,description,species,number_of_samples,is_archival,clearing_progress,clearer user in the lab,link_to_clearing_spreadsheet,imaging_progress calculated attribute,imager calculated attribute,total_imaging_requests calculated attribute,imaging_request_number calculated attribute,processing_request_number calculated attribute,processor calculated attribute,processing_progress calculated attribute,total_processing_requests calculated attribute
abondy,A297,A297-001,1,reconstruction of probe tracks (two Neuropixels probes dipped in CM-DiI). no need for single-cell resolution.,rat,1,0,in progress,ll3,None,incomplete,None,1,1,1,abondy,incomplete,1


## Test insert into imagingchannel for abondy A294 request

In [5]:
imaging_channel_contents = db_lightsheet.Request.ImagingChannel() & {'request_name':'A294','channel_name':'488'}
imaging_channel_contents

username user in the lab,request_name,sample_name,imaging_request_number,image_resolution,channel_name,ventral_up whether brain was flipped upside down to be imaged,imaging_date,zoom_body_magnification only applicable for 2x,left_lightsheet_used,right_lightsheet_used,registration,injection_detection,probe_detection,cell_detection,generic_imaging,pixel_type,image_orientation how the imager imaged the sample. Most of the time will be horizontal,numerical_aperture it is not always recorded in metadata so those times it will be NULL,tiling_scheme,tiling_overlap,z_step distance between z planes in microns,number_of_z_planes,rawdata_subfolder,"imspector_channel_index refers to multi-channel imaging - 0 if first (or only) channel in rawdata_subfolder, 1 if second, 2 if third, ...",left_lightsheet_precomputed_spock_jobid,left_lightsheet_precomputed_spock_job_progress,right_lightsheet_precomputed_spock_jobid,right_lightsheet_precomputed_spock_job_progress
abondy,A294,A294-001,1,3.6x,488,0,None,1.0,1,1,0,0,0,0,0,None,horizontal,nan,4x8,0.1,2.0,900,Ex_488_Em_0,0,None,None,None,None


In [8]:
imaging_channel_dict= imaging_channel_contents.fetch1()
imaging_channel_dict['z_step']=10
db_lightsheet.Request.ImagingChannel().insert1(imaging_channel_dict,replace=True)

## Test main dash route with datajoint version 0.13

In [ ]:
clearing_batch_contents = db_lightsheet.Request.ClearingBatch()
request_contents = db_lightsheet.Request()
sample_contents = db_lightsheet.Request.Sample()
requests_samples_joined = request_contents * sample_contents    
combined_contents = (clearing_batch_contents * request_contents).proj(
    'number_in_batch','expected_handoff_date',
    'clearing_protocol','species',
    'clearer','clearing_progress','clearing_protocol','antibody1','antibody2',
    datetime_submitted='TIMESTAMP(date_submitted,time_submitted)')

In [ ]:
contents_being_cleared = combined_contents & 'clearing_progress="in progress"'
contents_ready_to_clear = combined_contents & 'clearing_progress="incomplete"' 
contents_already_cleared = (combined_contents & 'clearing_progress="complete"')

In [ ]:
imaging_batch_contents = db_lightsheet.Request.ImagingBatch()
imaging_resolution_contents = db_lightsheet.Request.ImagingResolutionRequest()

imaging_request_contents = (clearing_batch_contents * sample_contents * \
    request_contents * imaging_batch_contents).\
    proj('clearer','clearing_progress',
    'imaging_request_date_submitted','imaging_request_time_submitted',
    'imaging_progress','imager','species','number_in_imaging_batch',
    datetime_submitted='TIMESTAMP(imaging_request_date_submitted,imaging_request_time_submitted)',)

In [ ]:
imaging_request_contents

In [ ]:
aggr_imaging_resolution_table = dj.U('username','request_name',
                                  'sample_name','imaging_request_number').aggr(
    imaging_resolution_contents,
    all_samples_lavision='SUM(IF(microscope="lavision",1,0))=count(*)',
    all_samples_smartspim='SUM(IF(microscope="smartspim",1,0))=count(*)')

In [ ]:
joined_table = imaging_request_contents*aggr_imaging_resolution_table
joined_table

In [ ]:
combined_imaging_contents = dj.U('username','request_name','sample_name',
    'clearing_batch_number',
    'imaging_batch_number','imaging_request_number').aggr(
    joined_table,
    clearer='MIN(clearer)',
    clearing_progress='MIN(clearing_progress)',
    imaging_progress='MIN(imaging_progress)',
    all_samples_cleared='SUM(IF(clearing_progress="complete",1,0))=count(*)')
combined_imaging_contents

In [ ]:
contents_being_imaged = combined_imaging_contents & 'all_samples_cleared=1' & \
    'imaging_progress="in progress"'

''' Next get all entities that are ready to be imaged '''
contents_ready_to_image = combined_imaging_contents & 'all_samples_cleared=1' & \
 'imaging_progress="incomplete"'
contents_on_deck = combined_imaging_contents & 'clearing_progress!="complete"' & 'imaging_progress!="complete"'
''' Finally get all entities that have already been imaged '''
contents_already_imaged = (combined_imaging_contents & 'imaging_progress="complete"')

x_imaging = {
    'Ready': len(contents_ready_to_image),
    'In progress': len(contents_being_imaged),
    'Imaged': len(contents_already_imaged),
}

## Figuring out all_requests route upgrades from dj version 0.12 to 0.13

In [ ]:
dj.__version__

In [11]:
request_contents = db_lightsheet.Request()
clearing_batch_contents = db_lightsheet.Request.ClearingBatch()
imaging_request_contents = db_lightsheet.Request.ImagingRequest()
processing_request_contents = db_lightsheet.Request.ProcessingRequest()
# logger.debug(request_contents)
''' Now figure out what fraction of the samples in each request are cleared/imaged/processed '''    
replicated_args = dict(is_archival='is_archival',number_of_samples='number_of_samples',description='description',
    species='species',datetime_submitted='datetime_submitted')

request_joined_contents = dj.U('username','request_name').aggr(
    request_contents * clearing_batch_contents,
    is_archival='MIN(is_archival)',
    number_of_samples='MIN(number_of_samples)',
    number_in_batch='MIN(number_in_batch)',
    description='MIN(description)',
    species='MIN(species)',
    datetime_submitted='TIMESTAMP(MIN(date_submitted),MIN(time_submitted))',
    n_cleared='CONVERT(SUM(IF(clearing_progress="complete",number_in_batch,0)),char)').proj(
    **replicated_args,
        fraction_cleared='CONCAT(n_cleared,"/",CONVERT(number_of_samples,char))')

# imaging_joined_contents = sample_joined_contents.aggr(
#     imaging_request_contents,
#     is_archival='MIN(is_archival)',
#     number_of_samples='MIN(number_of_samples)',
#     description='MIN(description)',
#     species='MIN(species)',
#     datetime_submitted='MIN(datetime_submitted)',
#     fraction_cleared='MIN(fraction_cleared)',
#     n_imaged='CONVERT(SUM(imaging_progress="complete"),char)',
#     total_imaging_requests='CONVERT(COUNT(*),char)',
#     keep_all_rows=True
#     ).proj(**replicated_args,
#         fraction_cleared='fraction_cleared',
#         # fraction_imaged='CONCAT(n_imaged,"/",total_imaging_requests)'
#         fraction_imaged='IF(n_imaged is NULL,"0/0",CONCAT(n_imaged,"/",total_imaging_requests))' 
#         )
# logger.debug("Imaging joined contents:")
# logger.debug(imaging_joined_contents)
# processing_joined_contents = (dj.U('username','request_name') * imaging_joined_contents).aggr(   
# processing_request_contents,
# **replicated_args,
# fraction_cleared='MIN(fraction_cleared)',
# fraction_imaged='MIN(fraction_imaged)',
# n_processed='CONVERT(SUM(processing_progress="complete"),char)',
# total_processing_requests='CONVERT(COUNT(processing_progress),char)',
# keep_all_rows=True
# ).proj(
#     **replicated_args,
#     fraction_cleared='fraction_cleared',
#     fraction_imaged='fraction_imaged',
#     fraction_processed='IF(n_processed is NULL,"0/0",CONCAT(n_processed,"/",total_processing_requests))' 
#     )

In [13]:
imaging_aggr_contents = dj.U('username','request_name').aggr(
    imaging_request_contents,
    n_imaged='CONVERT(SUM(imaging_progress="complete"),char)',
    total_imaging_requests='CONVERT(COUNT(*),char)',
    )
imaging_aggr_contents

username user in the lab,request_name,n_imaged calculated attribute,total_imaging_requests calculated attribute
abondy,A294,0,1
abondy,A323,1,1
abondy,TS_CTB_3_rats,3,3
afalkner,"MFNP2,_MFNP3,_MMNP4,_MMNP5,_MMNP6,_FMNP4,_FMNP5,_FMNP6",8,8
aichen,zimmerman_01_LaVision,1,1
apv2,20190313_IBL_DiI_1,1,1
apv2,ibl_witten_04,1,1
cz15,zimmerman_01,2,2
cz15,zimmerman_02,25,25
cz15,zimmerman_03,0,2


In [16]:
imaging_joined_contents = (request_joined_contents*imaging_aggr_contents).proj(**replicated_args,
        fraction_cleared='fraction_cleared',
        fraction_imaged='IF(n_imaged is NULL,"0/0",CONCAT(n_imaged,"/",total_imaging_requests))' 
        )

In [20]:
processing_aggr_contents = dj.U('username','request_name').aggr(
    processing_request_contents,
    n_processed='CONVERT(SUM(processing_progress="complete"),char)',
    total_processing_requests='CONVERT(COUNT(processing_progress),char)',
    )

In [25]:
processing_joined_contents = (imaging_joined_contents*processing_aggr_contents).proj(
        **replicated_args,
        fraction_cleared='fraction_cleared',
        fraction_imaged='fraction_imaged',
        fraction_processed='IF(n_processed is NULL,"0/0",CONCAT(n_processed,"/",total_processing_requests))' 
        ) 

username user in the lab,request_name,is_archival calculated attribute,number_of_samples calculated attribute,description calculated attribute,species calculated attribute,datetime_submitted calculated attribute,fraction_cleared calculated attribute,fraction_imaged calculated attribute,fraction_processed calculated attribute
ysagiv,ys_seqpc3_fp,0,4,Localisation of fibre tips,mouse,2021-07-08 14:28:09,4/4,4/4,0/4


In [28]:
request_contents = db_lightsheet.Request()
clearing_batch_contents = db_lightsheet.Request.ClearingBatch()
imaging_request_contents = db_lightsheet.Request.ImagingRequest()
processing_request_contents = db_lightsheet.Request.ProcessingRequest()
# logger.debug(request_contents)
''' Now figure out what fraction of the samples in each request are cleared/imaged/processed '''    
replicated_args = dict(is_archival='is_archival',number_of_samples='number_of_samples',description='description',
    species='species',datetime_submitted='datetime_submitted')

request_joined_contents = dj.U('username','request_name').aggr(
    request_contents * clearing_batch_contents,
    is_archival='MIN(is_archival)',
    number_of_samples='MIN(number_of_samples)',
    number_in_batch='MIN(number_in_batch)',
    description='MIN(description)',
    species='MIN(species)',
    datetime_submitted='TIMESTAMP(MIN(date_submitted),MIN(time_submitted))',
    n_cleared='CONVERT(SUM(IF(clearing_progress="complete",number_in_batch,0)),char)').proj(
    **replicated_args,
        fraction_cleared='CONCAT(n_cleared,"/",CONVERT(number_of_samples,char))')

imaging_aggr_contents = dj.U('username','request_name').aggr(
    imaging_request_contents,
    n_imaged='CONVERT(SUM(imaging_progress="complete"),char)',
    total_imaging_requests='CONVERT(COUNT(*),char)',
    )
imaging_joined_contents = (request_joined_contents*imaging_aggr_contents).proj(**replicated_args,
    fraction_cleared='fraction_cleared',
    fraction_imaged='IF(n_imaged is NULL,"0/0",CONCAT(n_imaged,"/",total_imaging_requests))' 
    )
processing_aggr_contents = dj.U('username','request_name').aggr(
    processing_request_contents,
    n_processed='CONVERT(SUM(processing_progress="complete"),char)',
    total_processing_requests='CONVERT(COUNT(processing_progress),char)',
    )
processing_joined_contents = (imaging_joined_contents*processing_aggr_contents).proj(
    **replicated_args,
    fraction_cleared='fraction_cleared',
    fraction_imaged='fraction_imaged',
    fraction_processed='IF(n_processed is NULL,"0/0",CONCAT(n_processed,"/",total_processing_requests))' 
    ) 

## Getting request_overview to work with datajoint 0.13

In [39]:
username='ysagiv'
request_name = "ys_seqpc3_fp"

request_contents = db_lightsheet.Request() & f'request_name="{request_name}"' & \
        f'username="{username}"'
request_contents = request_contents.proj('description','species','number_of_samples',
    'is_archival',datetime_submitted='TIMESTAMP(date_submitted,time_submitted)')
sample_contents = db_lightsheet.Request.Sample() & f'request_name="{request_name}"' & f'username="{username}"' 

clearing_batch_contents = db_lightsheet.Request.ClearingBatch() & \
    f'request_name="{request_name}"' & f'username="{username}"' 
clearing_batch_sample_contents = db_lightsheet.Request.ClearingBatchSample() & \
    f'request_name="{request_name}"' & f'username="{username}"' 
imaging_request_contents = db_lightsheet.Request.ImagingRequest() & \
    f'request_name="{request_name}"' & f'username="{username}"' 
processing_request_contents = db_lightsheet.Request.ProcessingRequest() & \
    f'request_name="{request_name}"' & f'username="{username}"' 

replicated_args = dict(number_of_samples='number_of_samples',description='description',
    species='species',clearing_progress='clearing_progress',
    clearer='clearer',
    imaging_progress='imaging_progress',imager='imager',
    is_archival='is_archival',
    link_to_clearing_spreadsheet='link_to_clearing_spreadsheet')

replicated_args_aggr = dict(number_of_samples='MIN(number_of_samples)',description='MIN(description)',
    species='MIN(species)',clearing_progress='MIN(clearing_progress)',
    clearer='MIN(clearer)',
    imaging_progress='MIN(imaging_progress)',imager='MIN(imager)',
    is_archival='MIN(is_archival)',
    link_to_clearing_spreadsheet='MIN(link_to_clearing_spreadsheet)')

sample_joined_contents = request_contents * clearing_batch_sample_contents * clearing_batch_contents.proj(
'clearing_progress','clearer','link_to_clearing_spreadsheet')

imaging_joined_contents = sample_joined_contents.aggr(
    imaging_request_contents,
    **replicated_args,
    imaging_request_number='imaging_request_number',
    n_imaged='CONVERT(SUM(imaging_progress="complete"),char)',
    total_imaging_requests='COUNT(*)',
    keep_all_rows=True
    ).proj(**replicated_args,
           total_imaging_requests='IF(n_imaged is NULL, "0",total_imaging_requests)',
           imaging_request_number='IF(imaging_request_number is NULL, "N/A",imaging_request_number)'
        # fraction_imaged='CONCAT(n_imaged,"/",total_imaging_requests)'
        )
processing_joined_contents = (dj.U('username','request_name') * imaging_joined_contents).aggr(   
    processing_request_contents,
    **replicated_args,
    imaging_request_number='imaging_request_number',
    processing_request_number='processing_request_number',
    processor='processor',
    processing_progress='processing_progress',
    total_imaging_requests='total_imaging_requests',
    n_processed='CONVERT(SUM(processing_progress="complete"),char)',
    total_processing_requests='CONVERT(COUNT(processing_progress),char)',
    keep_all_rows=True
    ).proj(
        **replicated_args,
        processing_request_number='IF(processing_request_number is NULL, "N/A",processing_request_number)',
        processor='processor',
        processing_progress='processing_progress',
        total_imaging_requests='total_imaging_requests',
        total_processing_requests='IF(n_processed is NULL,0,total_processing_requests)', 
        )

In [53]:
imaging_aggr_contents = dj.U('username','request_name','sample_name').aggr(
    imaging_request_contents,
    imaging_progress='MIN(imaging_progress)',
    imager='MIN(imager)',
    imaging_request_number='MIN(imaging_request_number)',
    total_imaging_requests='COUNT(*)',
    n_imaged='CONVERT(SUM(imaging_progress="complete"),char)',).proj(**replicated_args,
           total_imaging_requests='IF(n_imaged is NULL, "0",total_imaging_requests)',
           imaging_request_number='IF(imaging_request_number is NULL, "N/A",imaging_request_number)')
imaging_aggr_contents

username user in the lab,request_name,sample_name,imaging_progress calculated attribute,imager calculated attribute,total_imaging_requests calculated attribute,imaging_request_number calculated attribute
ysagiv,ys_seqpc3_fp,ys_seqpc3_fp-001,complete,None,1,1
ysagiv,ys_seqpc3_fp,ys_seqpc3_fp-002,complete,None,1,1
ysagiv,ys_seqpc3_fp,ys_seqpc3_fp-003,complete,None,1,1
ysagiv,ys_seqpc3_fp,ys_seqpc3_fp-004,complete,None,1,1


In [54]:
imaging_joined_contents = sample_joined_contents * imaging_aggr_contents

In [55]:
imaging_joined_contents

username user in the lab,request_name,sample_name,clearing_batch_number,description,species,number_of_samples,is_archival,datetime_submitted calculated attribute,clearing_protocol,antibody1,antibody2,clearing_progress,clearer user in the lab,link_to_clearing_spreadsheet,imaging_progress calculated attribute,imager calculated attribute,total_imaging_requests calculated attribute,imaging_request_number calculated attribute
ysagiv,ys_seqpc3_fp,ys_seqpc3_fp-001,1,Localisation of fibre tips,mouse,4,0,2021-07-08 14:28:09,iDISCO abbreviated clearing,,,complete,ll3,None,complete,None,1,1
ysagiv,ys_seqpc3_fp,ys_seqpc3_fp-002,1,Localisation of fibre tips,mouse,4,0,2021-07-08 14:28:09,iDISCO abbreviated clearing,,,complete,ll3,None,complete,None,1,1
ysagiv,ys_seqpc3_fp,ys_seqpc3_fp-003,1,Localisation of fibre tips,mouse,4,0,2021-07-08 14:28:09,iDISCO abbreviated clearing,,,complete,ll3,None,complete,None,1,1
ysagiv,ys_seqpc3_fp,ys_seqpc3_fp-004,1,Localisation of fibre tips,mouse,4,0,2021-07-08 14:28:09,iDISCO abbreviated clearing,,,complete,ll3,None,complete,None,1,1


In [70]:
processing_aggr_contents = dj.U('username','request_name').aggr(   
    processing_request_contents,
    processing_request_number='MIN(processing_request_number)',
    processor='MIN(processor)',
    processing_progress='MIN(processing_progress)',
    n_processed='CONVERT(SUM(processing_progress="complete"),char)',
    total_processing_requests='CONVERT(COUNT(processing_progress),char)')

In [71]:
processing_joined_contents = (imaging_joined_contents * processing_aggr_contents).proj(
    **replicated_args,
    processing_request_number='IF(processing_request_number is NULL, "N/A",processing_request_number)',
    processor='processor',
    processing_progress='processing_progress',
    total_imaging_requests='total_imaging_requests',
    total_processing_requests='IF(n_processed is NULL,0,total_processing_requests)', 
)

In [72]:
processing_joined_contents

username user in the lab,request_name,sample_name,clearing_batch_number,description,species,number_of_samples,is_archival,clearing_progress,clearer user in the lab,link_to_clearing_spreadsheet,imaging_progress calculated attribute,imager calculated attribute,total_imaging_requests calculated attribute,processor calculated attribute,processing_progress calculated attribute,processing_request_number calculated attribute,total_processing_requests calculated attribute
ysagiv,ys_seqpc3_fp,ys_seqpc3_fp-001,1,Localisation of fibre tips,mouse,4,0,complete,ll3,None,complete,None,1,ahoag,running,1,4
ysagiv,ys_seqpc3_fp,ys_seqpc3_fp-002,1,Localisation of fibre tips,mouse,4,0,complete,ll3,None,complete,None,1,ahoag,running,1,4
ysagiv,ys_seqpc3_fp,ys_seqpc3_fp-003,1,Localisation of fibre tips,mouse,4,0,complete,ll3,None,complete,None,1,ahoag,running,1,4
ysagiv,ys_seqpc3_fp,ys_seqpc3_fp-004,1,Localisation of fibre tips,mouse,4,0,complete,ll3,None,complete,None,1,ahoag,running,1,4


In [61]:
all_contents_dict_list = processing_joined_contents.fetch(as_dict=True)


## Updating all_samples route to datajoint version 0.13

In [74]:
request_contents = db_lightsheet.Request()
request_contents = request_contents.proj('description','species','number_of_samples',
    'is_archival','auditor',
    datetime_submitted='TIMESTAMP(date_submitted,time_submitted)')
sample_contents = db_lightsheet.Request.Sample()
clearing_batch_contents = db_lightsheet.Request.ClearingBatch()
imaging_request_contents = db_lightsheet.Request.ImagingRequest()
processing_request_contents = db_lightsheet.Request.ProcessingRequest()

replicated_args = dict(number_of_samples='number_of_samples',description='description',
    species='species',datetime_submitted='datetime_submitted',
    clearer='clearer',
    clearing_progress='clearing_progress',
    imager='imager',imaging_progress='imaging_progress',
    is_archival='is_archival',
    link_to_clearing_spreadsheet='link_to_clearing_spreadsheet')

sample_joined_contents = request_contents * sample_contents * clearing_batch_contents

imaging_joined_contents = sample_joined_contents.aggr(
    imaging_request_contents,
    **replicated_args,
    imaging_request_number='imaging_request_number',
    n_imaged='CONVERT(SUM(imaging_progress="complete"),char)',
    total_imaging_requests='COUNT(*)',
    keep_all_rows=True
    ).proj(**replicated_args,
           total_imaging_requests='IF(n_imaged is NULL, "0",total_imaging_requests)',
           imaging_request_number='IF(imaging_request_number is NULL, "N/A",imaging_request_number)'
        # fraction_imaged='CONCAT(n_imaged,"/",total_imaging_requests)'
        )
processing_joined_contents = (dj.U('username','request_name') * imaging_joined_contents).aggr(   
    processing_request_contents,
    **replicated_args,
    imaging_request_number='imaging_request_number',
    processing_request_number='processing_request_number',
    processor='processor',
    processing_progress='processing_progress',
    total_imaging_requests='total_imaging_requests',
    n_processed='CONVERT(SUM(processing_progress="complete"),char)',
    total_processing_requests='CONVERT(COUNT(processing_progress),char)',
    keep_all_rows=True
    ).proj(
        **replicated_args,
        processing_request_number='IF(processing_request_number is NULL, "N/A",processing_request_number)',
        processor='processor',
        processing_progress='processing_progress',
        total_imaging_requests='total_imaging_requests',
        total_processing_requests='IF(n_processed is NULL,0,total_processing_requests)', 
        )


In [86]:
imaging_joined_contents = sample_joined_contents * imaging_request_contents 

In [92]:
processing_joined_contents = imaging_joined_contents * processing_request_contents & {'request_name':'zimmerman_02'}

username user in the lab,request_name,sample_name,clearing_batch_number,imaging_request_number,processing_request_number,auditor user in the lab,description,species,number_of_samples,is_archival,datetime_submitted calculated attribute,subject_fullname,clearing_protocol,antibody1,antibody2,antibody1_lot,antibody2_lot,clearing_progress,number_in_batch,perfusion_date,expected_handoff_date,clearer user in the lab,notes_for_clearer,link_to_clearing_spreadsheet,imager user in the lab,imaging_request_date_submitted date that the user submitted the request for imaging,imaging_request_time_submitted time that the user submitted the request for imaging,imaging_performed_date date that the imaging form was submitted by the imager,imaging_progress,processor user in the lab,processing_request_date_submitted date that the user submitted the request for processing,processing_request_time_submitted time that the user submitted the request for processing,processing_performed_date date that the processing form was submitted by the processor,processing_progress
cz15,zimmerman_02,zimmerman_02-f11,1,1,1,None,"Clearing, Fos staining, and smartSPIM imaging for twelve brains.",mouse,12,0,2021-03-03 15:53:40,,iDISCO+_immuno,"as before: Synaptic Systems rabbit anti-Fos, 1:1000","as before: 647 donkey anti-rabbit, 1:200",Synaptic Systems lot 9-89,Invitrogen 2181018,complete,12,2021-03-01,2021-03-03,ll3,Please use the same clearing parameters/protocol as for the Nov 2020 pilot brains. (for sample: zimmerman_02-f11) Please use the same clearing parameters/protocol as for the Nov 2020 pilot brains. (for sample: zimmerman_02-f12) Please use the same clearing parameters/protocol as for the Nov 2020 pilot brains. (for sample: zimmerman_02-f13) Please use the same clearing parameters/protocol as for the Nov 2020 pilot brains. (for sample: zimmerman_02-f21) Please use the same clearing parameters/protocol as for the Nov 2020 pilot brains. (for sample: zimmerman_02-f22) Please use the same clearing parameters/protocol as for the Nov 2020 pilot brains. (for sample: zimmerman_02-f23) Please use the same clearing parameters/protocol as for the Nov 2020 pilot brains. (for sample: zimmerman_02-m11) Please use the same clearing parameters/protocol as for the Nov 2020 pilot brains. (for sample: zimmerman_02-m12) Please use the same clearing parameters/protocol as for the Nov 2020 pilot brains. (for sample: zimmerman_02-m13) Please use the same clearing parameters/protocol as for the Nov 2020 pilot brains. (for sample: zimmerman_02-m21) Please use the same clearing parameters/protocol as for the Nov 2020 pilot brains. (for sample: zimmerman_02-m22) Please use the same clearing parameters/protocol as for the Nov 2020 pilot brains. (for sample: zimmerman_02-m23),None,None,2021-03-03,15:53:40,2021-04-18,complete,cz15,2021-03-03,15:53:40,None,incomplete
cz15,zimmerman_02,zimmerman_02-f11,1,2,1,None,"Clearing, Fos staining, and smartSPIM imaging for twelve brains.",mouse,12,0,2021-03-03 15:53:40,,iDISCO+_immuno,"as before: Synaptic Systems rabbit anti-Fos, 1:1000","as before: 647 donkey anti-rabbit, 1:200",Synaptic Systems lot 9-89,Invitrogen 2181018,complete,12,2021-03-01,2021-03-03,ll3,Please use the same clearing parameters/protocol as for the Nov 2020 pilot brains. (for sample: zimmerman_02-f11) Please use the same clearing parameters/protocol as for the Nov 2020 pilot brains. (for sample: zimmerman_02-f12) Please use the same clearing parameters/protocol as for the Nov 2020 pilot brains. (for sample: zimmerman_02-f13) Please use the same clearing parameters/protocol as for the Nov 2020 pilot brains. (for sample: zimmerman_02-f21) Please use the same clearing parameters/protocol as for the Nov 2020 pilot brains. (for sample: zimmerman_02-f22) Please use the same clearing parameters/protocol as for the Nov 2020 pilot brains. (for sample: zimmerman_02-f23) Please use the same clearing parameters/protocol as for the Nov 2020 pilot brains. (for sample: zimmerman_02-m11) Plea

In [84]:
dj.U('username','request_name','imaging_request_number').aggr(
    imaging_request_contents)

username user in the lab,request_name,imaging_request_number
abondy,A294,1
abondy,A323,1
abondy,TS_CTB_3_rats,1
afalkner,"MFNP2,_MFNP3,_MMNP4,_MMNP5,_MMNP6,_FMNP4,_FMNP5,_FMNP6",1
aichen,zimmerman_01_LaVision,1
apv2,20190313_IBL_DiI_1,1
apv2,ibl_witten_04,1
cz15,zimmerman_01,1
cz15,zimmerman_02,1
cz15,zimmerman_02,2


In [85]:
#imaging_aggr_contents = 
dj.U('username','request_name','imaging_request_number').aggr(
    imaging_request_contents,
    n_imaged='CONVERT(SUM(imaging_progress="complete"),char)',
    total_imaging_requests='MAX(imaging_request_number)',
    ) 

username user in the lab,request_name,imaging_request_number,n_imaged calculated attribute,total_imaging_requests calculated attribute
abondy,A294,1,0,1
abondy,A323,1,1,1
abondy,TS_CTB_3_rats,1,3,1
afalkner,"MFNP2,_MFNP3,_MMNP4,_MMNP5,_MMNP6,_FMNP4,_FMNP5,_FMNP6",1,8,1
aichen,zimmerman_01_LaVision,1,1,1
apv2,20190313_IBL_DiI_1,1,1,1
apv2,ibl_witten_04,1,1,1
cz15,zimmerman_01,1,2,1
cz15,zimmerman_02,1,12,1
cz15,zimmerman_02,2,11,2


## Updating imaging_manager route to datajoint v 0.13

In [6]:
request_contents = db_lightsheet.Request()
sample_contents = db_lightsheet.Request.Sample()
clearing_batch_contents = db_lightsheet.Request.ClearingBatch()
imaging_batch_contents = db_lightsheet.Request.ImagingBatch()
imaging_resolution_contents = db_lightsheet.Request.ImagingResolutionRequest()

sample_joined_contents = clearing_batch_contents * sample_contents * \
    request_contents
clearing_aggr_contents = dj.U('username','request_name',
    'clearing_batch_number').aggr(
    sample_joined_contents,
    clearing_progress='MAX(clearing_progress)',
    clearer='MIN(clearer)',
    species='MIN(species)',
    all_samples_cleared='SUM(IF(clearing_progress="complete",1,0))=count(*)',
    )

imaging_aggr_contents = dj.U('username','request_name',
                             'imaging_request_number','imaging_batch_number').aggr(
    imaging_batch_contents * imaging_resolution_contents,
    imaging_request_date_submitted='MIN(imaging_request_date_submitted)',
    imaging_request_time_submitted='MIN(imaging_request_time_submitted)',
    all_samples_lavision='SUM(IF(microscope="lavision",1,0))=count(*)',
    all_samples_smartspim='SUM(IF(microscope="smartspim",1,0))=count(*)').proj(
    datetime_submitted='TIMESTAMP(imaging_request_date_submitted,imaging_request_time_submitted)',
    all_samples_lavision='all_samples_lavision',
    all_samples_smartspim='all_samples_smartspim'
)
imaging_joined_contents = (imaging_batch_contents * imaging_aggr_contents)
all_joined_contents = clearing_aggr_contents * imaging_joined_contents


In [7]:
all_joined_contents & 'clearing_progress!="complete"' & 'imaging_progress!="complete"'

username user in the lab,request_name,clearing_batch_number,imaging_batch_number,imaging_request_number,clearing_progress calculated attribute,clearer calculated attribute,species calculated attribute,all_samples_cleared calculated attribute,imager user in the lab,number_in_imaging_batch date that the imaging form was submitted by the imager,imaging_request_date_submitted date that the user submitted the request for imaging,imaging_request_time_submitted time that the user submitted the request for imaging,imaging_performed_date date that the imaging form was submitted by the imager,imaging_progress,imaging_dict,all_samples_lavision calculated attribute,all_samples_smartspim calculated attribute,datetime_submitted calculated attribute
cz15,zimmerman_04,1,1,1,in progress,ll3,mouse,0,cz15,5,2021-07-20,14:32:30,None,incomplete,=BLOB=,0,1,2021-07-20 14:32:30
cz15,zimmerman_05,1,1,1,in progress,ll3,mouse,0,cz15,12,2021-07-23,14:34:31,None,incomplete,=BLOB=,0,1,2021-07-23 14:34:31
fr1510,africanstripedmousepilot01,1,1,1,in progress,ll3,mouse,0,None,2,2021-07-31,16:30:24,None,incomplete,=BLOB=,0,1,2021-07-31 16:30:24
fr1510,africanstripedmousepilot01,2,1,1,in progress,ll3,mouse,0,None,2,2021-07-31,16:30:24,None,incomplete,=BLOB=,0,1,2021-07-31 16:30:24
ll3,Clearing_Reversal_Pilot_2_iDISCOplus,1,1,1,in progress,ll3,mouse,0,None,2,2021-07-21,14:59:05,None,incomplete,=BLOB=,0,1,2021-07-21 14:59:05
mkislin,Ai148_2p_mapping_PNIA_and_PLCb4Titration,1,1,1,in progress,ll3,mouse,0,None,1,2021-07-15,23:03:54,None,incomplete,=BLOB=,0,1,2021-07-15 23:03:54
mkislin,Ai148_2p_mapping_PNIA_and_PLCb4Titration,2,1,1,in progress,ll3,mouse,0,None,1,2021-07-15,23:03:54,None,incomplete,=BLOB=,0,1,2021-07-15 23:03:54
mkislin,Ai148_2p_mapping_PNIA_and_PLCb4Titration,3,1,1,in progress,ll3,mouse,0,None,1,2021-07-15,23:03:54,None,incomplete,=BLOB=,0,1,2021-07-15 23:03:54


In [5]:
all_joined_contents & 'all_samples_cleared=1' & \
'imaging_progress="in progress"'

username user in the lab,request_name,clearing_batch_number,imaging_batch_number,imaging_request_number,clearing_progress calculated attribute,species calculated attribute,all_samples_cleared calculated attribute,imager user in the lab,number_in_imaging_batch date that the imaging form was submitted by the imager,imaging_request_date_submitted date that the user submitted the request for imaging,imaging_request_time_submitted time that the user submitted the request for imaging,imaging_performed_date date that the imaging form was submitted by the imager,imaging_progress,imaging_dict,all_samples_lavision calculated attribute,all_samples_smartspim calculated attribute,datetime_submitted calculated attribute
jverpeut,092220_vectorandcrusIbil_openfieldonly,1,1,2,complete,mouse,1,sg3271,20,2021-04-26,14:34:24,None,in progress,=BLOB=,0,1,2021-04-26 14:34:24


In [150]:
all_joined_contents & 'all_samples_cleared=1' & \
     'imaging_progress="incomplete"'

username user in the lab,request_name,clearing_batch_number,imaging_batch_number,imaging_request_number,clearing_progress calculated attribute,species calculated attribute,all_samples_cleared calculated attribute,imager user in the lab,number_in_imaging_batch date that the imaging form was submitted by the imager,imaging_request_date_submitted date that the user submitted the request for imaging,imaging_request_time_submitted time that the user submitted the request for imaging,imaging_performed_date date that the imaging form was submitted by the imager,imaging_progress,imaging_dict,all_samples_lavision calculated attribute,all_samples_smartspim calculated attribute,datetime_submitted calculated attribute
cz15,zimmerman_03,1,1,1,complete,mouse,1,cz15,2,2021-07-20,14:27:24,None,incomplete,=BLOB=,0,1,2021-07-20 14:27:24


In [151]:
all_joined_contents & 'clearing_progress!="complete"' & 'imaging_progress!="complete"'

username user in the lab,request_name,clearing_batch_number,imaging_batch_number,imaging_request_number,clearing_progress calculated attribute,species calculated attribute,all_samples_cleared calculated attribute,imager user in the lab,number_in_imaging_batch date that the imaging form was submitted by the imager,imaging_request_date_submitted date that the user submitted the request for imaging,imaging_request_time_submitted time that the user submitted the request for imaging,imaging_performed_date date that the imaging form was submitted by the imager,imaging_progress,imaging_dict,all_samples_lavision calculated attribute,all_samples_smartspim calculated attribute,datetime_submitted calculated attribute
cz15,zimmerman_04,1,1,1,in progress,mouse,0,cz15,5,2021-07-20,14:32:30,None,incomplete,=BLOB=,0,1,2021-07-20 14:32:30
cz15,zimmerman_05,1,1,1,in progress,mouse,0,cz15,12,2021-07-23,14:34:31,None,incomplete,=BLOB=,0,1,2021-07-23 14:34:31
fr1510,africanstripedmousepilot01,1,1,1,in progress,mouse,0,None,2,2021-07-31,16:30:24,None,incomplete,=BLOB=,0,1,2021-07-31 16:30:24
fr1510,africanstripedmousepilot01,2,1,1,in progress,mouse,0,None,2,2021-07-31,16:30:24,None,incomplete,=BLOB=,0,1,2021-07-31 16:30:24
ll3,Clearing_Reversal_Pilot_2_iDISCOplus,1,1,1,in progress,mouse,0,None,2,2021-07-21,14:59:05,None,incomplete,=BLOB=,0,1,2021-07-21 14:59:05
mkislin,Ai148_2p_mapping_PNIA_and_PLCb4Titration,1,1,1,in progress,mouse,0,None,1,2021-07-15,23:03:54,None,incomplete,=BLOB=,0,1,2021-07-15 23:03:54
mkislin,Ai148_2p_mapping_PNIA_and_PLCb4Titration,2,1,1,in progress,mouse,0,None,1,2021-07-15,23:03:54,None,incomplete,=BLOB=,0,1,2021-07-15 23:03:54
mkislin,Ai148_2p_mapping_PNIA_and_PLCb4Titration,3,1,1,in progress,mouse,0,None,1,2021-07-15,23:03:54,None,incomplete,=BLOB=,0,1,2021-07-15 23:03:54


## Updating pystripe_manager route for datajoint v0.13

In [183]:
imaging_request_contents = db_lightsheet.Request.ImagingRequest() 
imaging_channel_contents = db_lightsheet.Request.ImagingChannel() 
imaged_channel_contents = imaging_channel_contents * imaging_request_contents &\
    {'imaging_progress':'complete'} & 'image_resolution="3.6x"'

stitching_contents = db_lightsheet.Request.SmartspimStitchedChannel()

imaged_aggr_contents = dj.U('username','request_name','sample_name',
    'imaging_request_number').aggr(
    imaged_channel_contents,
    n_channels_imaged='count(*)')

stitched_aggr_contents = dj.U('username','request_name','sample_name',
    'imaging_request_number').aggr(
    stitching_contents,
    n_channels_stitched='sum(smartspim_stitching_spock_job_progress="COMPLETED")')

imaged_and_stitched_contents = imaged_aggr_contents * stitched_aggr_contents &\
    'n_channels_imaged=n_channels_stitched'

pystripe_channel_contents = db_lightsheet.Request.SmartspimPystripeChannel() 

pystripe_channel_aggr_contents = dj.U('username','request_name','sample_name','imaging_request_number').aggr(
    pystripe_channel_contents,n_channels_pystriped='SUM(pystripe_performed)',
    n_channels_started='COUNT(*)',)

final_combined_contents = imaged_and_stitched_contents*pystripe_channel_aggr_contents



In [186]:
(imaged_and_stitched_contents - pystripe_channel_contents).fetch(as_dict=True)

[{'username': 'jverpeut',
  'request_name': '092220_vectorandcrusIbil_openfieldonly',
  'sample_name': '092220_vectorandcrusIbil_openfieldonly-001',
  'imaging_request_number': 2,
  'n_channels_imaged': 2,
  'n_channels_stitched': Decimal('2')},
 {'username': 'jverpeut',
  'request_name': '092220_vectorandcrusIbil_openfieldonly',
  'sample_name': '092220_vectorandcrusIbil_openfieldonly-003',
  'imaging_request_number': 2,
  'n_channels_imaged': 2,
  'n_channels_stitched': Decimal('2')},
 {'username': 'jverpeut',
  'request_name': '092220_vectorandcrusIbil_openfieldonly',
  'sample_name': '092220_vectorandcrusIbil_openfieldonly-004',
  'imaging_request_number': 2,
  'n_channels_imaged': 2,
  'n_channels_stitched': Decimal('2')},
 {'username': 'jverpeut',
  'request_name': '092220_vectorandcrusIbil_openfieldonly',
  'sample_name': '092220_vectorandcrusIbil_openfieldonly-005',
  'imaging_request_number': 2,
  'n_channels_imaged': 2,
  'n_channels_stitched': Decimal('2')},
 {'username': 'j

In [184]:
final_combined_contents &\
    'n_channels_pystriped!=n_channels_imaged' & 'n_channels_started>0' 

username user in the lab,request_name,sample_name,imaging_request_number,n_channels_imaged calculated attribute,n_channels_stitched calculated attribute,n_channels_pystriped calculated attribute,n_channels_started calculated attribute
abondy,A323,A323-001,1,2,2,0,2
abondy,TS_CTB_3_rats,A288,1,2,2,0,2
abondy,TS_CTB_3_rats,A292,1,2,2,0,2
abondy,TS_CTB_3_rats,A324,1,2,2,0,2
cz15,zimmerman_02,zimmerman_02-f11,1,2,2,0,2
cz15,zimmerman_02,zimmerman_02-f11,2,2,2,0,2
cz15,zimmerman_02,zimmerman_02-f12,1,2,2,0,2
cz15,zimmerman_02,zimmerman_02-f12,2,2,2,0,2
cz15,zimmerman_02,zimmerman_02-f13,1,2,2,0,2
cz15,zimmerman_02,zimmerman_02-f13,2,2,2,0,2


In [167]:
combined_contents

username user in the lab,request_name,sample_name,imaging_request_number,image_resolution,channel_name,ventral_up whether brain was flipped upside down to be imaged,n_channels_imaged calculated attribute,n_channels_stitched calculated attribute,flatfield_filename the name of the flat field file generated by the FlatGenerate GUI on smartspim computer,pystripe_performed whether pystripe has been run on this channel,datetime_pystripe_started,datetime_pystripe_completed,smartspim_pystripe_spock_jobid the jobid from running pystripe,smartspim_pystripe_spock_job_progress the spock job status code for pystripe,smartspim_corrected_precomputed_spock_jobid,smartspim_corrected_precomputed_spock_job_progress
abondy,A323,A323-001,1,3.6x,488,0,2,2,flat.tiff,0,2021-06-25 12:35:47,None,22252817,COMPLETED,22252848,COMPLETED
abondy,A323,A323-001,1,3.6x,642,0,2,2,flat.tiff,0,2021-06-24 19:55:13,None,22252174,COMPLETED,22252253,CANCELLED
abondy,TS_CTB_3_rats,A288,1,3.6x,488,0,2,2,flat.tiff,0,2021-07-09 12:54:27,None,22331841,COMPLETED,None,None
abondy,TS_CTB_3_rats,A288,1,3.6x,642,0,2,2,flat.tiff,0,2021-07-09 12:54:30,None,22331842,COMPLETED,None,None
abondy,TS_CTB_3_rats,A292,1,3.6x,488,0,2,2,flat.tiff,0,2021-07-09 12:54:40,None,22331843,COMPLETED,None,None
abondy,TS_CTB_3_rats,A292,1,3.6x,642,0,2,2,flat.tiff,0,2021-07-09 12:54:41,None,22331844,COMPLETED,None,None
abondy,TS_CTB_3_rats,A324,1,3.6x,488,0,2,2,flat.tiff,0,2021-07-09 12:54:48,None,22331845,COMPLETED,None,None
abondy,TS_CTB_3_rats,A324,1,3.6x,642,0,2,2,flat.tiff,0,2021-07-09 12:54:49,None,22331854,COMPLETED,None,None
cz15,zimmerman_01,zimmerman_01-001,1,3.6x,488,0,2,2,flat.tiff,1,None,None,None,None,None,None
cz15,zimmerman_01,zimmerman_01-001,1,3.6x,642,0,2,2,flat.tiff,1,None,None,None,None,None,None


In [166]:
dj.U('username','request_name','sample_name',
    'imaging_request_number').aggr(combined_contents,
                                   username='username',
                                   n_channels_imaged='n_channels_imaged',
                                   n_channels_pystriped='sum(pystripe_performed)',
                                   n_channels_started='count(*)',
                                  )

OperationalError: (1055, "'$47.n_channels_imaged' isn't in GROUP BY")

OperationalError: (1055, "'$49.n_channels_imaged' isn't in GROUP BY")

In [161]:
imaging_requests_currently_being_pystriped

OperationalError: (1055, "'$3f.n_channels_imaged' isn't in GROUP BY")

OperationalError: (1055, "'$41.n_channels_imaged' isn't in GROUP BY")

## Experimenting with update1() function in datajoint 0.13

In [95]:
clearing_batch_contents = db_lightsheet.Request.ClearingBatch() & {'request_name':'test_3p6x'}
clearing_batch_contents

username user in the lab,request_name,clearing_batch_number,clearing_protocol,antibody1,antibody2,antibody1_lot,antibody2_lot,clearing_progress,number_in_batch,perfusion_date,expected_handoff_date,clearer user in the lab,notes_for_clearer,link_to_clearing_spreadsheet
ahoag,test_3p6x,1,iDISCO abbreviated clearing,,,None,None,complete,1,None,None,ahoag,,None


In [102]:
update_dict = clearing_batch_contents.fetch1()
update_dict['antibody1_lot'] = 'test_antibody'
db_lightsheet.Request.ClearingBatch.update1(update_dict)

In [103]:
db_lightsheet.Request.ClearingBatch() & {'request_name':'test_3p6x'}

username user in the lab,request_name,clearing_batch_number,clearing_protocol,antibody1,antibody2,antibody1_lot,antibody2_lot,clearing_progress,number_in_batch,perfusion_date,expected_handoff_date,clearer user in the lab,notes_for_clearer,link_to_clearing_spreadsheet
ahoag,test_3p6x,1,iDISCO abbreviated clearing,,,test_antibody,None,complete,1,None,None,ahoag,,None


## Test updating a column in SmartspimStitchedChannel() using datajoint version 0.12

In [ ]:
jobid=22252860
this_stitching_content = db_lightsheet.Request.SmartspimStitchedChannel() & f'smartspim_stitching_spock_jobid={jobid}'

In [ ]:
this_stitching_content

In [ ]:
status_step3="COMPLETED"

In [ ]:
this_stitching_dict = this_stitching_content.fetch1()
replace_stitching_dict =  this_stitching_dict.copy()
replace_stitching_dict['smartspim_stitching_spock_job_progress'] = status_step3
replace_stitching_dict

In [ ]:
this_stitching_content

In [ ]:
this_stitching_content._update('smartspim_stitching_spock_job_progress','CANCELLED')

In [ ]:
this_stitching_content.fetch1()

In [ ]:
dj.config['safemode'] = False
this_stitching_content.delete_quick()

In [ ]:
db_lightsheet.Request.SmartspimStitchedChannel().insert1(replace_stitching_dict,replace=True)

In [ ]:
replace_stitching_dict

In [ ]:
db_lightsheet.Request.ImagingRequest() & {'request_name':'test_already_stitched'}

In [ ]:
db_lightsheet.Request.ImagingResolutionRequest() & {'request_name':'test_already_stitched'}

In [ ]:
db_lightsheet.Request.Sample() & {'request_name':'test_already_stitched'}

In [ ]:
db_lightsheet.Request.ImagingBatch() & {'username':'ahoag'}

In [ ]:
db_lightsheet.Request.ImagingChannel() & {'username': 'ahoag', 'request_name': 'test_already_stitched', 'sample_name': 'test_already_stitched-001', 'imaging_request_number': 1, 'image_resolution': '3.6x', 'channel_name': '488', 'ventral_up': False}

In [ ]:
db_lightsheet.Request.SmartspimStitchedChannel()

In [ ]:
db_lightsheet.Request.ClearingBatch() & {'username':'hboele'}

In [ ]:
db_lightsheet.Request.ClearingBatchSample() & {'username':'hboele'}

In [ ]:
db_lightsheet.Request.IdiscoPlusImmunoClearing()

In [ ]:
db_lightsheet.Request.ImagingBatchSample() & {'username':'hboele'}

In [ ]:
job_contents = db_spockadmin.ProcessingPipelineSpockJob()
unique_contents = dj.U('jobid_step0','username',).aggr(
    job_contents,timestamp='max(timestamp)')*job_contents

In [ ]:
unique_contents

In [ ]:
request_contents = db_lightsheet.Request()
clearing_batch_contents = request_contents.ClearingBatch()
combined_contents = request_contents *clearing_batch_contents

In [ ]:
combined_contents

In [ ]:
username='soline'

In [ ]:
db_lightsheet.Request.ImagingChannel() & {'username':username}

In [ ]:
db_lightsheet.Request.ProcessingResolutionRequest() & {'username':username}

In [ ]:
db_lightsheet.Request.ImagingChannel() & {'username':username}

In [ ]:
request_contents = db_lightsheet.Request()
clearing_batch_contents = db_lightsheet.Request.ClearingBatch()
imaging_request_contents = db_lightsheet.Request.ImagingRequest()
processing_request_contents = db_lightsheet.Request.ProcessingRequest()

In [ ]:
restrict_OR_list = [{"username":username},{"auditor":username}]
request_contents = request_contents & restrict_OR_list
imaging_request_contents = (request_contents*imaging_request_contents & restrict_OR_list).proj('imager',
                              'imaging_request_date_submitted',
                              'imaging_request_time_submitted',
                              'imaging_performed_date',
                              'imaging_progress')
processing_request_contents = (request_contents*processing_request_contents & restrict_OR_list).proj(
                            'processing_progress',
                            'processor')

In [ ]:
replicated_args = dict(is_archival='is_archival',number_of_samples='number_of_samples',description='description',
        species='species',datetime_submitted='datetime_submitted')

In [ ]:
sample_joined_contents = dj.U('username','request_name').aggr(
        request_contents * clearing_batch_contents,
        is_archival='is_archival',
        number_of_samples='number_of_samples',
        number_in_batch='number_in_batch',
        description='description',
        species='species',
        datetime_submitted='TIMESTAMP(date_submitted,time_submitted)',
        n_cleared='CONVERT(SUM(IF(clearing_progress="complete",number_in_batch,0)),char)').proj(
        **replicated_args,
            fraction_cleared='CONCAT(n_cleared,"/",CONVERT(number_of_samples,char))')

In [ ]:
imaging_joined_contents = sample_joined_contents.aggr(
    imaging_request_contents,
    **replicated_args,
    fraction_cleared='fraction_cleared',
    n_imaged='CONVERT(SUM(imaging_progress="complete"),char)',
    total_imaging_requests='CONVERT(COUNT(*),char)',
    keep_all_rows=True
    ).proj(**replicated_args,
        fraction_cleared='fraction_cleared',
        # fraction_imaged='CONCAT(n_imaged,"/",total_imaging_requests)'
        fraction_imaged='IF(n_imaged is NULL,"0/0",CONCAT(n_imaged,"/",total_imaging_requests))' 
        )

In [ ]:
processing_joined_contents = (dj.U('username','request_name') * imaging_joined_contents).aggr(   
    processing_request_contents,
    **replicated_args,
    fraction_cleared='fraction_cleared',
    fraction_imaged='fraction_imaged',
    n_processed='CONVERT(SUM(processing_progress="complete"),char)',
    total_processing_requests='CONVERT(COUNT(processing_progress),char)',
    keep_all_rows=True
    ).proj(
        **replicated_args,
        fraction_cleared='fraction_cleared',
        fraction_imaged='fraction_imaged',
        fraction_processed='IF(n_processed is NULL,"0/0",CONCAT(n_processed,"/",total_processing_requests))' 
        )

In [ ]:
processing_joined_contents

## Debugging all samples with auditor

In [ ]:
request_contents = db_lightsheet.Request()
request_contents = request_contents.proj('description','species','number_of_samples',
        'is_archival','auditor',
        datetime_submitted='TIMESTAMP(date_submitted,time_submitted)')
sample_contents = db_lightsheet.Request.Sample()
clearing_batch_contents = db_lightsheet.Request.ClearingBatch()
imaging_request_contents = db_lightsheet.Request.ImagingRequest()
processing_request_contents = db_lightsheet.Request.ProcessingRequest()

restrict_OR_list = [{"username":username},{"auditor":username}]
request_contents = request_contents & restrict_OR_list
sample_contents = (request_contents*sample_contents & restrict_OR_list).proj()

clearing_batch_contents = (request_contents*clearing_batch_contents & restrict_OR_list).proj(
    'clearer','clearing_progress','link_to_clearing_spreadsheet')
imaging_request_contents = (request_contents*imaging_request_contents & restrict_OR_list).proj(
    'imager','imaging_progress')
processing_request_contents = (request_contents*processing_request_contents & restrict_OR_list).proj(
    'processor','processing_progress')
sample_joined_contents = request_contents * sample_contents * clearing_batch_contents
replicated_args = dict(number_of_samples='number_of_samples',description='description',
        species='species',datetime_submitted='datetime_submitted',
        clearer='clearer',
        clearing_progress='clearing_progress',
        imager='imager',imaging_progress='imaging_progress',
        is_archival='is_archival',
        link_to_clearing_spreadsheet='link_to_clearing_spreadsheet')

In [ ]:
imaging_joined_contents = sample_joined_contents.aggr(
        imaging_request_contents,
        **replicated_args,
        imaging_request_number='imaging_request_number',
        n_imaged='CONVERT(SUM(imaging_progress="complete"),char)',
        total_imaging_requests='COUNT(*)',
        keep_all_rows=True
        ).proj(**replicated_args,
               total_imaging_requests='IF(n_imaged is NULL, "0",total_imaging_requests)',
               imaging_request_number='IF(imaging_request_number is NULL, "N/A",imaging_request_number)'
            # fraction_imaged='CONCAT(n_imaged,"/",total_imaging_requests)'
            )

In [ ]:
processing_joined_contents = (dj.U('username','request_name') * imaging_joined_contents).aggr(   
        processing_request_contents,
        **replicated_args,
        imaging_request_number='imaging_request_number',
        processing_request_number='processing_request_number',
        processor='processor',
        processing_progress='processing_progress',
        total_imaging_requests='total_imaging_requests',
        n_processed='CONVERT(SUM(processing_progress="complete"),char)',
        total_processing_requests='CONVERT(COUNT(processing_progress),char)',
        keep_all_rows=True
        ).proj(
            **replicated_args,
            processing_request_number='IF(processing_request_number is NULL, "N/A",processing_request_number)',
            processor='processor',
            processing_progress='processing_progress',
            total_imaging_requests='total_imaging_requests',
            total_processing_requests='IF(n_processed is NULL,0,total_processing_requests)', 
            )

In [ ]:
processing_joined_contents

In [ ]:
contents = db_lightsheet.Request() & [{"username":username},{"auditor":username}]
contents

In [ ]:
imaging_batch_restrict_dict = dict(username=username,request_name=request_name,
    imaging_batch_number=imaging_batch_number,
    imaging_request_number=imaging_request_number)
sample_contents = db_lightsheet.Request.ImagingBatchSample() & imaging_batch_restrict_dict

In [ ]:
sample_contents

In [ ]:
restrict_dict = {'username':'lightserv-test','request_name':'test_new_imaging_request'}
db_lightsheet.Request() & restrict_dict

In [ ]:
db_lightsheet.Request.ImagingRequest & restrict_dict

In [ ]:
db_lightsheet.Request.ImagingBatch() & restrict_dict

In [ ]:
request_contents = db_lightsheet.Request()
sample_contents = db_lightsheet.Request.Sample()
clearing_batch_contents = db_lightsheet.Request.ClearingBatch()
imaging_batch_contents = db_lightsheet.Request.ImagingBatch()

imaging_request_contents = (clearing_batch_contents * sample_contents * \
    request_contents * imaging_batch_contents).\
    proj('clearer','clearing_progress',
    'imaging_request_date_submitted','imaging_request_time_submitted',
    'imaging_progress','imager','species','number_in_imaging_batch',
    datetime_submitted='TIMESTAMP(imaging_request_date_submitted,imaging_request_time_submitted)')

In [ ]:
imaging_request_contents = dj.U('username','request_name',
                                'imaging_batch_number',
                                'imaging_request_number').aggr(
    imaging_request_contents,clearer='clearer',
    clearing_progress='clearing_progress',
    datetime_submitted='datetime_submitted',
    imaging_progress='imaging_progress',imager='imager',
    species='species',number_in_imaging_batch='number_in_imaging_batch',
    all_samples_cleared='SUM(IF(clearing_progress="complete",1,0))=count(*)')

In [ ]:
imaging_request_contents & restrict_dict

In [ ]:
imaging_request_contents & restrict_dict

In [ ]:
contents_being_imaged = imaging_request_contents & 'all_samples_cleared=1' & \
    'imaging_progress="in progress"'

In [ ]:
contents_being_imaged & restrict_dict

In [ ]:
imaging_request_contents & 'imaging_request_number=2'

In [ ]:
sample_restrict_dict = \
    {'sample_name': 'test-001', 'imaging_request_number': '2', 'imaging_batch_number': '1'}

In [ ]:
db_lightsheet.Request.ClearingBatchSample()

In [ ]:
(db_lightsheet.Request() & 'username="pbibawi"').fetch('request_name')

In [ ]:
username='willmore'
request_name='201106_willmore'
sample_name = '201106_willmore-303'
imaging_request_number=1
processing_request_number=1
processing_restrict_dict = dict(username=username,request_name=request_name,
                sample_name=sample_name,imaging_request_number=imaging_request_number,
                processing_request_number=processing_request_number)

processing_request_contents = db_lightsheet.Request.ProcessingRequest() & processing_restrict_dict

processing_resolution_request_contents = db_lightsheet.Request.ProcessingResolutionRequest() & \
    processing_restrict_dict

processing_channel_contents = db_lightsheet.Request.ProcessingChannel() & processing_restrict_dict
image_resolution='1.1x'
processing_channel_contents_this_resolution = (processing_channel_contents & \
            f'image_resolution="{image_resolution}"')
restrict_dict = {'channel_name':'488','ventral_up':0}
this_processing_channel_content = (processing_channel_contents_this_resolution & \
                    restrict_dict)
this_processing_channel_content

In [ ]:
username='willmore'
request_name='201106_willmore'
imaging_batch_number=1
imaging_request_number=1

restrict_dict = dict(username=username,request_name=request_name,
                imaging_batch_number=imaging_batch_number)

channel_contents_all_samples = (db_lightsheet.Request.ImagingChannel() & f'request_name="{request_name}"' & \
    f'username="{username}"' )
imaging_request_contents = db_lightsheet.Request.ImagingRequest() & \
    restrict_dict
imaging_resolution_contents = db_lightsheet.Request.ImagingResolutionRequest() & \
    restrict_dict
imaging_resolution_contents

In [ ]:
channel_contents_all_samples

In [ ]:
image_resolutions_no_processing = ["2x","3.6x","15x"]

In [ ]:
batch_unique_image_resolutions = set(imaging_resolution_contents.fetch('image_resolution'))
# batch_unique_image_re?solutions = set(['1.1x','3.6x'])

In [ ]:
have_processing_requests = any([x not in image_resolutions_no_processing for x in batch_unique_image_resolutions])
have_processing_requests

In [ ]:
set(imaging_request_contents.fetch('sample_name'))


In [ ]:
sample_names = set(imaging_resolution_contents.fetch('sample_name'))
sample_names

In [ ]:
data_rootpath = '/jukebox/LightSheetData/lightserv'

In [ ]:
message_body = ('Hello!\n\nThis is an automated email sent from lightserv, '
                        'the Light Sheet Microscopy portal at the Histology and Brain Registration Core Facility. '
                        'The raw data your request:\n'
                        f'request_name: "my-request"\n'
                        f'are now available on bucket here: \n\n')
for sample_name in sample_names:
    sample_basepath = os.path.join(data_rootpath,username,request_name,
    sample_name,f'imaging_request_number_{imaging_request_number}',  
    'rawdata')
    channel_contents_this_sample = channel_contents_all_samples & f'sample_name="{sample_name}"' 
    message_body += f'Sample name: {sample_name}:\n'
    for channel_dict in channel_contents_this_sample:
        channel_name = channel_dict['channel_name']
        image_resolution = channel_dict['image_resolution']
        ventral_up = channel_dict['ventral_up']
        rawdata_subfolder = channel_dict['rawdata_subfolder']
        if ventral_up:
            rawdata_fullpath = os.path.join(
                data_rootpath,username,request_name,sample_name,
                f'imaging_request_{imaging_request_number}',
                'rawdata',f'resolution_{image_resolution}_ventral_up',
                rawdata_subfolder)
            message_body += f'\tchannel {channel_name} (ventral up): {rawdata_fullpath}\n'
        else:
            rawdata_fullpath = os.path.join(
                data_rootpath,username,request_name,sample_name,
                f'imaging_request_{imaging_request_number}',
                'rawdata',f'resolution_{image_resolution}',
                rawdata_subfolder)
            message_body += f'\tchannel {channel_name}: {rawdata_fullpath}\n'
    message_body += '\n'
message_body += '\n'
if have_processing_requests:
    message_body += ('To start processing your data, '
            f'go to the processing management GUI: /some/path '
            'and find your sample to process.\n\n')
message_body += 'Thanks,\n\nThe Core Facility'
print(message_body)

In [ ]:
this_processing_channel_content_dict = this_processing_channel_content.fetch1()
this_processing_channel_content_dict

In [ ]:
processing_channel_contents_this_resolution

In [ ]:
db_lightsheet.Request.ProcessingResolutionRequest().fetch('atlas_name',limit=1)[0]

In [ ]:
processing_resolution_request_contents

In [ ]:
request_contents = db_lightsheet.Request()
sample_contents = db_lightsheet.Request.Sample()
clearing_batch_contents = db_lightsheet.Request.ClearingBatch()
imaging_request_contents = db_lightsheet.Request.ImagingRequest()
imaging_channel_contents = db_lightsheet.Request.ImagingChannel()

In [ ]:
imaging_channel_contents & 'ventral_up=1'

In [ ]:
processing_resolution_contents = db_lightsheet.Request.ProcessingResolutionRequest() & {'username':'willmore'}
processing_resolution_contents 

In [ ]:
dj.Table._update(processing_resolution_contents,'atlas_name','allen_2017')

In [ ]:
clearing_batch_contents

In [ ]:
combined_contents = (clearing_batch_contents * request_contents).proj(
'description','number_in_batch','expected_handoff_date',
'clearing_protocol','species',
'clearer','clearing_progress','clearing_protocol','antibody1','antibody2',
datetime_submitted='TIMESTAMP(date_submitted,time_submitted)')
''' First get all entities that are currently being cleared '''

In [ ]:
idiscoplus_cleared_contents = combined_contents & 'clearing_protocol="iDISCO+_immuno"' & 'clearing_progress="complete"'
idiscoplus_cleared_contents

In [ ]:
df_idiscoplus_cleared = (idiscoplus_cleared_contents).fetch(
format='frame',order_by='datetime_submitted DESC')

In [ ]:
df_idiscoplus_cleared.to_csv("idiscoplus_already_cleared_asof_2020Nov12.csv")

In [ ]:
idisco_contents = db_lightsheet.Request.IdiscoPlusClearing()
idisco_contents

In [ ]:
idisco_website_cleared_contents = idisco_contents * idiscoplus_cleared_contents
idisco_website_cleared_contents

In [ ]:
idisco_results=idisco_website_cleared_contents.fetch(as_dict=True)
idisco_results

In [ ]:
df_idisco=pd.DataFrame(idisco_results)
df_idisco

In [ ]:
df_idisco.to_csv('idiscoplus_antibody_notes_already_cleared_asof_2020Nov12.csv')

In [ ]:
imaging_channel_contents & \
    {'username': 'ahoag',
     'request_name': 'test123',
     'sample_name': 'test123-001',
     'imaging_request_number': 1,
     'image_resolution': '1.3x',
     'channel_name': '488'}

In [ ]:
clearing_batch_contents.fetch(as_dict=True,limit=2)

In [ ]:
clearing_batch_contents.fetch?

In [ ]:
imaging_channel_contents & 'request_name="test_many_samples4"' 

In [ ]:
insert_dict = {'username': 'ahoag', 'request_name': 'test_many_samples4',
               'imaging_request_number': 1, 'image_resolution': '1.1x',
               'channel_name': '555',
               'injection_detection': 1,
               'sample_name': 'test_many_samples4-001'}


In [ ]:
imaging_merged_contents = (imaging_request_contents \
    * clearing_batch_contents * sample_contents * request_contents * imaging_channel_contents).\
    proj('clearer','clearing_progress',
    'imaging_request_date_submitted','imaging_request_time_submitted',
    'imaging_progress','imager','species',
    'image_resolution','channel_name',
    datetime_submitted='TIMESTAMP(imaging_request_date_submitted,imaging_request_time_submitted)') & \
    {'username':'ahoag',}

In [ ]:
replicated_args = dict(username='username',species='species',
                       datetime_submitted='datetime_submitted',
                       imager='imager',imaging_progress='imaging_progress')

In [ ]:
interm=dj.U("username","request_name","sample_name","imaging_request_number").aggr(
    imaging_merged_contents,new_col='GROUP_CONCAT(channel_name ORDER BY channel_name)',**replicated_args,
    image_resolution='image_resolution').\
    proj(comb_col='CONCAT(image_resolution,",",new_col)',**replicated_args,)
interm

In [ ]:
interm2=dj.U('request_name','sample_name','imaging_request_number').aggr(interm,
    **replicated_args,
    final_col='GROUP_CONCAT(comb_col)')
interm2

In [ ]:
dj.U('request_name','imaging_request_number','final_col').aggr(interm2,                               
    **replicated_args,
    number_in_batch='COUNT(*)')

In [ ]:
replicated_args = dict(username='username',
                       imager='imager',imaging_progress='imaging_progress')

In [ ]:
interm=dj.U("username","request_name","sample_name","imaging_request_number").aggr(
                        imaging_channel_contents,new_col='GROUP_CONCAT(channel_name ORDER BY channel_name)',
                        image_resolution='image_resolution').\
                        proj(comb_col='CONCAT(image_resolution,",",new_col)')
interm

In [ ]:
interm2=dj.U('request_name','sample_name','imaging_request_number').aggr(interm,
    sample_name='MIN(sample_name)',imaging_str='GROUP_CONCAT(comb_col)')
interm2

In [ ]:
interm3=dj.U('request_name','imaging_request_number','imaging_str').aggr(interm2,                               
    number_in_batch='COUNT(*)',).proj('request_name','number_in_batch')
interm3

In [ ]:
request_counter_dict = {}

In [ ]:
interm3.fetch(as_dict=True)

In [ ]:
imaging_channel_contents & 'request_name="test_many_samples3"'

In [ ]:
db_lightsheet.Request.ImagingChannel() & 'request_name="test_many_samples2"'

In [ ]:
db_lightsheet.Request.ImagingRequest() & 'request_name="test_many_samples2"'

In [ ]:
db_lightsheet.Request.ImagingResolutionRequest() & 'request_name="test_many_samples2"'

In [ ]:
db_lightsheet.Request.ImagingChannel() & 'request_name="test_many_samples2"'

In [ ]:
contents = db_lightsheet.Request.ImagingChannel() & 'request_name="test_many_samples2"'

In [ ]:
contents.fetch?

In [ ]:
db_lightsheet.Request.ImagingChannel().insert(channel_entry_dicts,skip_duplicates=True)

In [ ]:
db_lightsheet.Request.ImagingChannel() & 'request_name="test_many_samples2"'

In [ ]:
sorted(set(["a","b","a","a"]))

In [ ]:
db_lightsheet.Request.ProcessingResolutionRequest() & 'request_name="test_many_samples2"'

In [ ]:
db_lightsheet.Request.ProcessingChannel() & 'request_name="test_many_samples2"'

In [ ]:
contents 

In [ ]:
var=4
s = f'{{var}} is the number'
s

In [ ]:
b = '1.3x_ventral_up'

In [ ]:
b if 'ventral_up' not in b else b.strip("_ventral_up")

In [ ]:
b.strip("_ventral_up")

In [ ]:
l=[{'id':'A','image_resolution':'4x','ventral_up':True},
   {'id':'B','image_resolution':'4x','ventral_up':False},
    {'id':'C','image_resolution':'1.3x','ventral_up':False},
    {'id':'D','image_resolution':'1.3x','ventral_up':False},
    {'id':'E','image_resolution':'1.3x','ventral_up':True}
  ]
tup_list = [(d['image_resolution'],d['ventral_up']) for d in l]
sorted(set(tup_list))

In [ ]:
all_channel_contents = imaging_channel_contents & 'request_name="test_dorsal_ventral"'

In [ ]:
results_list=all_channel_contents.fetch('image_resolution','ventral_up',as_dict=True)
tup_list = [(d['image_resolution'],d['ventral_up']) for d in results_list]
set(tup_list)
# all_channel_contents.fetch

In [ ]:
for a,b, in tup_list:
    print(a,b)

In [ ]:
bool(int('0'))

In [ ]:
job_contents = db_spockadmin.ProcessingPipelineSpockJob()
job_contents

In [ ]:
unique_contents = dj.U('jobid_step0','username',).aggr(
		job_contents,timestamp='max(timestamp)')*job_contents
unique_contents

In [ ]:
int(False)

In [ ]:
int(bool("False"))

In [ ]:
bool("False")

In [ ]:
s='Invitrogen rabbit polyclonal aldolase C'
len(s)

In [ ]:
b='Invitrogen donkey anti rabbit AlexaFluor 647'
len(b)

In [ ]:
c='1:50, 1:100, 1:200'
len(c)

In [ ]:
d={
    'date': datetime.date(2021, 1, 25),
    'brief_descriptor': 'aldolase C titration',
    'animal_model': 'mouse',
    'primary_antibody': 'Invitrogen rabbit polyclonal aldolase C',
    'primary_concentration': '1:50, 1:100, 1:200',
    'primary_order_info': 'Invitrogen PA512317, lot V13073112', 'secondary_antibody': 'Invitrogen donkey anti rabbit AlexaFluor 647', 'secondary_concentration': '1:200', 'secondary_order_info': 'Invitrogen A-31573', 'notes': 'waiting for titration decision'}

In [ ]:
date = '2020-02-01'
datetime.strptime(date,'%Y-%m-%d')

In [ ]:
[1,2]+[3]

In [ ]:
sample_imaging_dict = {}

## Debugging request_overview 

In [ ]:
username='ahoag'
request_name='test_imaging_batches_clearing_batches'
request_contents = db_lightsheet.Request() & f'request_name="{request_name}"' & \
            f'username="{username}"'
request_contents = request_contents.proj('description','species','number_of_samples',
    'is_archival',datetime_submitted='TIMESTAMP(date_submitted,time_submitted)')
sample_contents = db_lightsheet.Request.Sample() & f'request_name="{request_name}"' & f'username="{username}"' 
clearing_batch_contents = db_lightsheet.Request.ClearingBatch() & \
f'request_name="{request_name}"' & f'username="{username}"'
clearing_batch_sample_contents = db_lightsheet.Request.ClearingBatchSample() & \
f'request_name="{request_name}"' & f'username="{username}"' 
imaging_request_contents = db_lightsheet.Request.ImagingRequest() & \
 f'request_name="{request_name}"' & f'username="{username}"' 
processing_request_contents = db_lightsheet.Request.ProcessingRequest() & \
 f'request_name="{request_name}"' & f'username="{username}"' 

In [ ]:
replicated_args = dict(number_of_samples='number_of_samples',description='description',
        species='species',clearing_progress='clearing_progress',
        clearer='clearer',
        imaging_progress='imaging_progress',imager='imager',
        is_archival='is_archival',
        link_to_clearing_spreadsheet='link_to_clearing_spreadsheet')

In [ ]:
sample_joined_contents = request_contents * clearing_batch_sample_contents * clearing_batch_contents.proj(
    'clearing_progress','clearer','link_to_clearing_spreadsheet')
imaging_joined_contents = sample_joined_contents.aggr(
        imaging_request_contents,
        **replicated_args,
        imaging_request_number='imaging_request_number',
        n_imaged='CONVERT(SUM(imaging_progress="complete"),char)',
        total_imaging_requests='COUNT(*)',
        keep_all_rows=True
        ).proj(**replicated_args,
               total_imaging_requests='IF(n_imaged is NULL, "0",total_imaging_requests)',
               imaging_request_number='IF(imaging_request_number is NULL, "N/A",imaging_request_number)'
            # fraction_imaged='CONCAT(n_imaged,"/",total_imaging_requests)'
            )
processing_joined_contents = (dj.U('username','request_name') * imaging_joined_contents).aggr(   
        processing_request_contents,
        **replicated_args,
        imaging_request_number='imaging_request_number',
        processing_request_number='processing_request_number',
        processor='processor',
        processing_progress='processing_progress',
        total_imaging_requests='total_imaging_requests',
        n_processed='CONVERT(SUM(processing_progress="complete"),char)',
        total_processing_requests='CONVERT(COUNT(processing_progress),char)',
        keep_all_rows=True
        ).proj(
            **replicated_args,
            processing_request_number='IF(processing_request_number is NULL, "N/A",processing_request_number)',
            processor='processor',
            processing_progress='processing_progress',
            total_imaging_requests='total_imaging_requests',
            total_processing_requests='IF(n_processed is NULL,0,total_processing_requests)', 
            )

In [ ]:
sample_joined_contents

In [ ]:
imaging_joined_contents

In [ ]:
processing_joined_contents

In [14]:
rawdata_subfolder="my bad path"
rawdata_subfolder = rawdata_subfolder.rstrip("/").strip()
rawdata_subfolder

'my bad path'

In [15]:
" " in rawdata_subfolder

True